# <center >OpenAI Agents SDK + MCP 智能体开发实战

## <center>Part 1. Agent & Runner 核心功能组件详解</center>

&emsp;&emsp;在《【加餐】OpenAI Agents SDK 开发实战》 的 1 ~ 8 小节课程中，我们已经初步介绍了关于`OpenAI Agents SDK` 的基本使用方法，主要包含了本地环境配置、不同大模型的在线API & 本地部署的接入方法，如何接入自定义工具函数等内容。作为一节快速入门的课程，很多`OpenAI Agents SDK`框架的应用细节并没有进行深入介绍，但实际上，各个功能的细节处理才是我们真正在工业环境下需要掌握的核心技巧。因此接下来的课程，我们就针对`OpenAI Agents SDK` 完整的底层逻辑和应用开发技巧展开详细的说明。


&emsp;&emsp;`OpenAI` 在多智能体搭建的尝试始于实验性质的 `Swarm` 框架，访问地址：https://github.com/openai/swarm ， 于2024年5月29日开源。这个框架的核心是希望简化多智能体工作流的创建，使开发人员能够设计能够进行任务委派和协作的智能体。但作为一个实验性的框架，`Swarm` 在可扩展性和稳健性方面存在局限性，因此更像是一种教育资源，而非一个可用于生产的解决方案。随着大模型基座模型的不断迭代，模型原生能力越来越强，因此`Multi-Agent` 的开发需求越来越迫切，因此，在2025年3月11号，`OpenAI`正式推出第一款企业级`Multi-Agent`开发框架`Agents-SDK`。 同时，在3月27号，`Agents SDK`正式官宣支持`MCP`使用，这也使得`Agents SDK`的实际应用场景得到极大的扩展。 `OpenAI Agents SDK` Github 开源地址:https://github.com/openai/openai-agents-python

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202505091156966.png" width=60%></div>

&emsp;&emsp;保持了`OpenAI`一贯的风格，虽然`Agents-SDK`框架是面向生产级的`Mutli-Agent`开发框架，但在应用开发时，仍然可以借助其比较高级的抽象类进行快速的原型开发。从整体架构上看，`Agents-SDK`由三大核心模块构成：Agent(代理)、Handoffs（交接）和Guardrails（护栏）。

- **核心组件一：Agent**

&emsp;&emsp;`OpenAI Agents-SDK`框架的的核心是代理，通过接入大模型构建自主的`AI`实体，用来执行任务、做出决策并与用户和其他代理无缝交互。例如，在客户服务场景中，代理可以处理咨询、处理订单并提供个性化建议，同时从每次交互中学习，以提升未来的绩效。

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202505091222491.png" width=60%></div>

- **核心组件二：Handoffs**

&emsp;&emsp;复杂的工作流程很难仅用一个代理包揽所有工作，而`Multi-Agent` 的架构形式在`OpenAI Agents SDK`中使用通过`Handoffs`组件来实现，它使代理能够将特定任务委托给其他专业代理。比如一个管理项目的 AI 代理识别出一项财务任务；它可以无缝地将此任务移交给财务专业代理，从而确保精准度和专业性。这种模块化方法反映了人类环境中有效的团队合作，在团队合作中，任务的分配基于个人优势。

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202505091249857.png" width=60%></div>




- **核心组件三：Guardrails**

&emsp;&emsp;护栏是 `AI Agent` 的安全机制，用于验证输入和输出，以确保代理在定义的参数范围内运行。对一些敏感信息，比如个人隐私、企业机密等，需要使用护栏进行保护，避免信息泄露。同时当把`AI Agent` 应用到实际场景中时，安全护栏也可以避免`AI Agent`自动执行高风险操作，比如删除数据、发布不当言论、虚假信息等。

&emsp;&emsp;三个核心组件的加持共同构建了`OpenAI Agents SDK`的完整框架，而如果更进一步了解其底层实现，如下图所示：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202505091334117.png" width=60%></div>


&emsp;&emsp;从图中可以看出，`OpenAI Agents SDK` 在运行时完整的生命周期主要包含以下几个核心组件的支撑：
- `Trace`：追踪代理的执行轨迹。
- `RunContext`：定义代理的执行上下文。
- `Agent`：定义代理的整体结构和具体行为。
- `Runner`：定义代理的运行方式。
- `Guardrail`：定义代理的安全机制，包括`InputGuardrail`和`OutputGuardrail`两种类型。
- `Tool`：定义代理的工具，这包括`Function Calling` 和 `MCP` 两种类型。
- `Handoff`：定义代理的交接行为。

&emsp;&emsp;通过这些核心组件的协同工作，`OpenAI Agents SDK` 可以构建出完整的`AI Agent` 框架，并应用到实际场景中。理解了上图中所有核心组件的实现细节，基本上就能够完全掌握`OpenAI Agents SDK` 框架的使用和工程化开发技巧。因此接下来的内容，我们就逐个组件开始进行详细的说明。


## 1. Agent 核心组件详解

&emsp;&emsp;Agents 组件是 Agents SDK 的核心組件之一。该组件用来定义代理的整体结构和具体行为。其源码位置为：https://github.com/openai/openai-agents-python/blob/main/src/agents/agent.py 。

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202505071518656.png" width=60%></div>

&emsp;&emsp; Agent 类通过`@dataclass` 装饰器定义，`@dataclass` 装饰器的主要作用是可以自动生成一个初始化方法（__init__），并根据类中的属性自动生成对应的参数。所以我们可以理解为`Agent`类的实例化是一个静态的过程，本质上是自定义一系列构建代理的参数。

```python
    @dataclass
    class Agent(Generic[TContext]):
        """An agent is an AI model configured with instructions, tools, guardrails, handoffs and more.

        We strongly recommend passing `instructions`, which is the "system prompt" for the agent. In
        addition, you can pass `handoff_description`, which is a human-readable description of the
        agent, used when the agent is used inside tools/handoffs.

        Agents are generic on the context type. The context is a (mutable) object you create. It is
        passed to tool functions, handoffs, guardrails, etc.
        """

        name: str
        """The name of the agent."""

        instructions: (
            str
            | Callable[
                [RunContextWrapper[TContext], Agent[TContext]],
                MaybeAwaitable[str],
            ]
            | None
        ) = None
```

&emsp;&emsp;我们抽象出来其主要参数如下：

<style>
.center 
{
  width: auto;
  display: table;
  margin-left: auto;
  margin-right: auto;
}
</style>

<p align="center"><font face="黑体" size=4>Agent 组件核心参数</font></p>
<div class="center">

| 属性名                | 类型                                                                                          | 描述                                                                                                   |
|---------------------|-----------------------------------------------------------------------------------------------|--------------------------------------------------------------------------------------------------------|
| <font color=red>name</font>             | `str`                                                                                         | 代理的名称。                                                                                             |
| <font color=red>instructions</font>      | `str` \| `Callable[[RunContextWrapper[TContext], Agent[TContext]], MaybeAwaitable[str]]` \| `None` | 代理的指令，用作“系统提示”。可以是字符串或动态生成指令的函数。                                           |
| `handoff_description` | `str` \| `None`                                                                              | 代理的描述，用于代理作为交接时，让 LLM 知道它的功能和何时调用它。                                         |
| `handoffs`          | list[Agent[Any] | Handoff[TContext]]                                                   | 代理可以委托的子代理列表。允许关注点分离和模块化。                                                       |
| `model`             | `str` \| `Model` \| `None`                                                                   | 调用 LLM 时使用的模型实现。默认情况下，如果未设置，代理将使用 `openai_provider.DEFAULT_MODEL` 中配置的默认模型。 |
| `model_settings`    | `ModelSettings`                                                                               | 配置模型特定的调优参数（例如温度、top_p）。                                                             |
| `tools`             | `list[Tool]`                                                                                 | 代理可以使用的工具列表。                                                                                 |
| `mcp_servers`       | `list[MCPServer]`                                                                             | 代理可以使用的模型上下文协议（MCP）服务器列表。                                                        |
| `mcp_config`        | `MCPConfig`                                                                                   | MCP 服务器的配置。                                                                                       |
| `input_guardrails`  | `list[InputGuardrail[TContext]]`                                                             | 在代理执行之前并行运行的检查列表，仅在代理是链中的第一个代理时运行。                                     |
| `output_guardrails` | `list[OutputGuardrail[TContext]]`                                                            | 在生成响应后对代理的最终输出运行的检查列表，仅在代理生成最终输出时运行。                                 |
| `output_type`       | type[Any] | AgentOutputSchemaBase | None                                                 | 输出对象的类型。如果未提供，输出将为 `str`。                                                             |
| `hooks`             | AgentHooks[TContext] | None                                                               | 接收代理生命周期事件回调的类。                                                                           |
| `tool_use_behavior` | Literal["run_llm_again", "stop_on_first_tool"] | StopAtTools | ToolsToFinalOutputFunction | 配置工具使用的处理方式。                                                                                 |
| `reset_tool_choice` | `bool`                                                                                       | 调用工具后是否将工具选择重置为默认值。默认为 `True`。确保代理不会进入工具使用的无限循环。                   |

&emsp;&emsp;这里我们先关注上表中标红的两个基本参数：

- name 指的是代理的名称，这个名称会用于在代理的执行过程中，作为代理的标识。
- instructions 指的是当前代理的系统信息，用来定义当前代理的职责、工作范围、工作目标等。其本质就是`system prompt`，其源码中核心代码逻辑如下：


```python
    async def get_system_prompt(self, run_context: RunContextWrapper[TContext]) -> str | None:
        """Get the system prompt for the agent."""
        if isinstance(self.instructions, str):
            return self.instructions
        elif callable(self.instructions):
            if inspect.iscoroutinefunction(self.instructions):
                return await cast(Awaitable[str], self.instructions(run_context, self))
            else:
                return cast(str, self.instructions(run_context, self))
        elif self.instructions is not None:
            logger.error(f"Instructions must be a string or a function, got {self.instructions}")

        return None
```

&emsp;&emsp;`Agent`组件的一个最关键因素就是如果将不同的模型作为其基座模型进行接入。因此，接下来我们首先进行不同大模型接入`OpenAI Agents SDK` 框架的实例应用。

### 1.1 OpenAI系列模型

&emsp;&emsp;`Agents-SDK`框架由`OpenAI`团队开发并开源，因此自然而然最适配的就是`OpenAI`自家的`GPT`系列模型。其配置和接入方法最简单，同时整体的兼容性也会更好。如果要接入`GPT`系列模型的话，首先需要依次在当前运行环境下安装必要的第三方依赖包：

In [1]:
# ! pip install openai python-dotenv openai-agents  # 如果需要安装，需要解开注释

In [1]:
import openai # type: ignore

openai.__version__

'1.77.0'

&emsp;&emsp;接下来在项目运行同级目录下手动新建一个`.env`文件中，填写`OpenAI`的有效`API_Key`，如下所示：

```markdown
    DEEPSEEK_API_KEY=sk-fb5a6d95f5cb59
    DEEPSEEK_BASE_URL=https://api.deepseek.com
    DEEPSEEK_MODEL=deepseek-chat

    QWEN3_API_KEY=sk-154d1e07ff
    QWEN3_BASE_URL=https://dashscope.aliyuncs.com/compatible-mode/v1
    QWEN3_MODEL="qwen3-235b-a22b"

    OPENAI_API_KEY=sk-proj-KV7yjKf9ZW8g_CzMuWoLGXo5GwULqxyQCNxk1jKmntbTtFHLpfTcU-_rkA
```

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202505071611714.png" width=60%></div>

&emsp;&emsp;使用`load_dotenv`从`.env`文件中加载` OpenAI API_Key`，执行如下代码：

In [2]:
import os
from dotenv import load_dotenv

load_dotenv(override=True) # 如果已经配置过全局变量，则使用.env 文件中的变量覆盖替换

True

In [4]:
# 读取OpenAI API-KEY
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# print(OPENAI_API_KEY)

&emsp;&emsp;接下来，便可以直接导入`Agent`模块，传入必要的参数进行代理的实例化，如下代码所示：

In [5]:
from agents import Agent

agent = Agent(
    name="乐于助人的私人小助理",
    instructions="请使用中文回答用户的问题",
)

&emsp;&emsp;默认情况下，`OpenAI SDK Agent` 框架在实例化`Agent`会时会查找 `OPENAI_API_KEY` 这个环境变量，所以当仅通过`name` 和 `instructions` 参数实例化`Agent`对象时，实例化的将是一个`gpt-4o`的模型实例。其源码定义位置：https://github.com/openai/openai-agents-python/blob/main/src/agents/models/openai_provider.py

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202505071627073.png" width=60%></div>

&emsp;&emsp;在《【加餐】OpenAI Agents SDK 开发实战》 的 1 ~ 8 小节课程中，已经介绍过当想要运行某个模型实例，在`OpenAI Agents SDK`框架下需要使用 `Runner` 这个核心组件，所以为了验证模型的连通性，我们先直接运行如下代码进行测试，在下一小节再展开介绍`Runner`模块的实现细节及应用技巧。

In [6]:
from agents import Runner

result = await Runner.run(
    starting_agent=agent,
    input="你好，请你介绍一下你自己"
    )

print(result.final_output)

你好！我是一个由OpenAI开发的人工智能助手，可以协助你解答各种问题、提供信息和帮助解决问题。如果你有任何问题或需要建议，请随时告诉我！


&emsp;&emsp;如果能正常收到大模型的响应，则说明模型接入成功。

### 1.2 其他在线&开源模型

&emsp;&emsp;`OpenAI Agents SDK` 除了可以使用`GPT`系列模型，同样也支持其他主流的如`DeepSeek v3 & r1`、`Qwen3`系列等在线API或本地部署的开源模型接入，其接入的规范主要包含以下两个方面：

1. 符合`OpenAI RESTFUL API`规范的在线API，比如 [DeepSeek官方API](https://platform.deepseek.com/usage), [阿里云百炼应用开发平台](https://bailian.console.aliyun.com/?spm=5176.29619931.J__Z58Z6CX7MY__Ll8p1ZOR.1.74cd521cmEK6EB&tab=home#/home)等；
2. 通过 `vLLM`、`Ollama` 等框架部署启动的模型，可以使用其`OpenAI`兼容的`REST API`地址实现接入；
3. 通过 `LiteLLM 框架` 集成的在线API & 本地部署的开源模型；

&emsp;&emsp;关于 `vLLM`、`Ollama`框架启动的开源模型如何接入，在《【加餐】OpenAI Agents SDK 开发实战》 的 1 ~ 8 小节课程中已经详细介绍过，这里不再重复说明。我们这里需要进一步扩展的是不同的模型接入方法其核心的配置参数应该如何使用。

&emsp;&emsp;首先，当我们需要接入非`OpenAI`的`GPT`系列模型时，需要进一步了解和掌握`Agent`组件如下两个标红的参数：

&emsp;&emsp;我们抽象出来其主要参数如下：

<style>
.center 
{
  width: auto;
  display: table;
  margin-left: auto;
  margin-right: auto;
}
</style>

<p align="center"><font face="黑体" size=4>Agent 组件核心参数</font></p>
<div class="center">

| 属性名                | 类型                                                                                          | 描述                                                                                                   |
|---------------------|-----------------------------------------------------------------------------------------------|--------------------------------------------------------------------------------------------------------|
| <font color=green>**name**</font>             | `str`                                                                                         | 代理的名称。                                                                                             |
| <font color=green>**instructions**</font>      | `str` \| `Callable[[RunContextWrapper[TContext], Agent[TContext]], MaybeAwaitable[str]]` \| `None` | 代理的指令，用作“系统提示”。可以是字符串或动态生成指令的函数。                                           |
| `handoff_description` | `str` \| `None`                                                                              | 代理的描述，用于代理作为交接时，让 LLM 知道它的功能和何时调用它。                                         |
| `handoffs`          | list[Agent[Any] | Handoff[TContext]]                                                   | 代理可以委托的子代理列表。允许关注点分离和模块化。                                                       |
| <font color=red>**model**</font>              | `str` \| `Model` \| `None`                                                                   | 调用 LLM 时使用的模型实现。默认情况下，如果未设置，代理将使用 `openai_provider.DEFAULT_MODEL` 中配置的默认模型。 |
| <font color=red>**model_settings**</font>    | `ModelSettings`                                                                               | 配置模型特定的调优参数（例如温度、top_p）。                                                             |
| `tools`             | `list[Tool]`                                                                                 | 代理可以使用的工具列表。                                                                                 |
| `mcp_servers`       | `list[MCPServer]`                                                                             | 代理可以使用的模型上下文协议（MCP）服务器列表。                                                        |
| `mcp_config`        | `MCPConfig`                                                                                   | MCP 服务器的配置。                                                                                       |
| `input_guardrails`  | `list[InputGuardrail[TContext]]`                                                             | 在代理执行之前并行运行的检查列表，仅在代理是链中的第一个代理时运行。                                     |
| `output_guardrails` | `list[OutputGuardrail[TContext]]`                                                            | 在生成响应后对代理的最终输出运行的检查列表，仅在代理生成最终输出时运行。                                 |
| `output_type`       | type[Any] | AgentOutputSchemaBase | None                                                 | 输出对象的类型。如果未提供，输出将为 `str`。                                                             |
| `hooks`             | AgentHooks[TContext] | None                                                               | 接收代理生命周期事件回调的类。                                                                           |
| `tool_use_behavior` | Literal["run_llm_again", "stop_on_first_tool"] | StopAtTools | ToolsToFinalOutputFunction | 配置工具使用的处理方式。                                                                                 |
| `reset_tool_choice` | `bool`                                                                                       | 调用工具后是否将工具选择重置为默认值。默认为 `True`。确保代理不会进入工具使用的无限循环。                   |

&emsp;&emsp;其中，`model`用来接收大模型的实例，包括必要的`api_key`、`baseurl`、`model_name`， 而`model_settings`则是可以用来定义大模型在生成响应时的采样参数，比如`temperature`、`top_k`等。这里我们使用`DeepSeek`的官方在线`API`进行测试。首先，需要在`.env`文件中正确的填写`DeepSeek`的有效`API_key`，如下图所示：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202505071703551.png" width=60%></div>

In [7]:
from openai import OpenAI
import os
from dotenv import load_dotenv
load_dotenv(override=True)

True

In [8]:
# 读取DeepSeek API-KEY
DEEPSEEK_API_KEY = os.getenv("DEEPSEEK_API_KEY")
DEEPSEEK_BASE_URL = os.getenv("DEEPSEEK_BASE_URL")
DEEPSEEK_MODEL = os.getenv("DEEPSEEK_MODEL")

print(DEEPSEEK_MODEL)
print(DEEPSEEK_BASE_URL)


deepseek-chat
https://api.deepseek.com


&emsp;&emsp;首先测试一下连通性：

In [9]:
# 实例化客户端
client = OpenAI(api_key=DEEPSEEK_API_KEY, 
                base_url=DEEPSEEK_BASE_URL)

response = client.chat.completions.create(
    model=DEEPSEEK_MODEL,
    messages=[
        {"role": "user", "content": "你好，好久不见!请介绍下你自己。"}
    ]
)

print(response.choices[0].message.content)

你好呀！很高兴再次相遇！😊 我是 **DeepSeek Chat**，由深度求索公司打造的智能AI助手。我的版本是 **DeepSeek-V3**，知识更新至 **2024年7月**，拥有 **128K 上下文记忆**，可以处理超长文本，还能阅读 **PDF、Word、Excel、PPT** 等文件来帮助你分析内容。  

✨ **我的特点**：  
- **免费使用**，没有隐藏收费！  
- **逻辑清晰**，擅长解答技术、学习、写作、编程等问题。  
- **支持超长对话**，可以深入讨论复杂话题。  
- **能联网搜索**（需你手动开启），获取最新信息。  

无论是学习、工作，还是日常生活中的疑问，都可以找我聊聊！最近有什么我可以帮你的吗？😃


&emsp;&emsp;如果能够正常生成回复，则说明模型服务正常。接下来我们尝试将其接入`OpenAI Agents SDK`框架中。`DeepSeek`符合`OpenAI RESTFUL API`规范，所以我们可以直接借助`OpenAIChatCompletionsModel`类来传递新的`API_Key`、`Base_url`等关键信息，其源码定义位置如下：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202505071715549.png" width=60%></div>

&emsp;&emsp;除此以外，`Runner.run`是一个异步方法，所以接入的模型实例也需要使用异步，因此这里实例化模型时需要使用`AsyncOpenAI`包，代码如下所示：

In [11]:
from openai import AsyncOpenAI

deepseek_client = AsyncOpenAI(
    base_url=DEEPSEEK_BASE_URL, 
    api_key=DEEPSEEK_API_KEY
    )

&emsp;&emsp;接下来在`Runner`中通过`model`参数进行传递，代码如下：

In [13]:
from agents import OpenAIChatCompletionsModel

agent = Agent(
    name="乐于助人的私人小助理",
    instructions="请使用中文回答用户的问题",
    model=OpenAIChatCompletionsModel(
        model=DEEPSEEK_MODEL,
        openai_client=deepseek_client,
    )
)

result = await Runner.run(agent, "你好，请你介绍一下你自己")
print(result.final_output)

你好！我是DeepSeek Chat，由深度求索公司研发的一款智能AI助手。我可以帮助你解答各种问题，包括学习、工作、编程、生活百科等多个领域。我具备强大的文本理解和生成能力，能够提供详细、准确的回答，并支持中文和英文交流。

我的特点包括：
1. **知识丰富**：掌握大量专业知识，覆盖科技、历史、文化、经济等多个领域。
2. **逻辑清晰**：能帮助分析问题、提供建议，甚至辅助代码编写和调试。
3. **长文本处理**：支持超长上下文（128K tokens），适合处理复杂内容。
4. **免费使用**：目前无需付费，你可以随时向我提问！  

无论是查资料、写作辅助，还是日常疑问，我都会尽力帮你解答。如果有任何问题，欢迎随时问我！😊


&emsp;&emsp;除此以外，还可以通过 `model_settings` 参数来配置模型在生成响应时的采样参数，比如温度、top_p等，其源码定义位置如下所示：https://github.com/openai/openai-agents-python/blob/main/src/agents/model_settings.py。 这里我们整理出具体可传递的参数说明如下：

<style>
.center 
{
  width: auto;
  display: table;
  margin-left: auto;
  margin-right: auto;
}
</style>

<p align="center"><font face="黑体" size=4>Agent 组件中model_seetings参数说明</font></p>
<div class="center">

| 属性名                | 类型                                                                                          | 描述                                                                                                   |
|---------------------|-----------------------------------------------------------------------------------------------|--------------------------------------------------------------------------------------------------------|
| `temperature`       | float                                                                                        | 调用模型时使用的温度。控制生成文本的随机性。较高的值（如 0.8）会使输出更随机，而较低的值（如 0.2）会使输出更确定。 |
| `top_p`             | float                                                                                        | 调用模型时使用的 top_p 值。控制生成文本的多样性。top_p 采样会考虑概率分布中前 p 的词，较低的值会使输出更集中。 |
| `frequency_penalty` | float                                                                                        | 调用模型时使用的频率惩罚。用于减少重复词的使用，值越高，重复的词越少。                                   |
| `presence_penalty`  | float                                                                                        | 调用模型时使用的存在惩罚。用于增加新词的使用，值越高，模型越倾向于使用新词而不是重复的词。               |
| `tool_choice`       | Literal["auto", "required", "none"]                                                         | 调用模型时使用的工具选择。定义模型在生成响应时是否需要使用工具。                                       |
| `parallel_tool_calls` | bool                                                                                       | 调用模型时是否使用并行工具调用。默认为 `False`，如果为 `True`，则可以同时调用多个工具。                 |
| `truncation`        | Literal["auto", "disabled"]                                                                  | 调用模型时使用的截断策略。定义如何处理超出最大令牌数的输入。                                           |
| `max_tokens`        | int                                                                                         | 生成的最大输出令牌数。限制模型生成的响应长度。                                                         |
| `reasoning`         | Reasoning                                                                                    | 推理模型的配置选项。用于配置推理模型的特定参数。                                                       |
| `metadata`          | dict[str, str]                                                                               | 与模型响应调用一起包含的元数据。可以用于跟踪请求或提供额外信息。                                       |
| `store`             | bool                                                                                         | 是否存储生成的模型响应以供后续检索。默认为 `True`，表示响应会被存储。                                   |
| `include_usage`     | bool                                                                                         | 是否包含使用块。默认为 `True`，表示响应中会包含使用信息。                                             |
| `extra_query`       | Query                                                                                        | 请求时提供的额外查询字段。默认为 `None`，可以用于传递额外的查询参数。                                   |
| `extra_body`        | Body                                                                                         | 请求时提供的额外主体字段。默认为 `None`，可以用于传递额外的请求体内容。                               |
| `extra_headers`     | Headers                                                                                      | 请求时提供的额外头部字段。默认为 `None`，可以用于传递额外的 HTTP 头部信息。                           |


&emsp;&emsp;自定义采样参数的方法，则是需要通过`ModelSettings`类进行实例化并传递，如下代码所示：

In [14]:
from agents import ModelSettings

agent = Agent(
    name="乐于助人的私人小助理",
    instructions="请使用中文回答用户的问题",
    model=OpenAIChatCompletionsModel(
        model=DEEPSEEK_MODEL,
        openai_client=deepseek_client,
    ),
    model_settings=ModelSettings(
        temperature=0.6,
        max_tokens=10,
    )
)
result = await Runner.run(agent, "请你介绍一下你自己")
print(result.final_output)

你好！我是DeepSeek Chat，由深度


&emsp;&emsp;此时可以看到，因为我们在模型设置中设置了 max_tokens=10，所以生成的结果只有10个字符，以此说明模型设置的参数是生效的，关于更多的参数设置，大家可以自行尝试。

### 1.3 使用LiteLLM集成模型

&emsp;&emsp;除了上述直接通过`OpenAIChatCompletionsModel`类接入符合`OpenAI RESTFUL API`规范的大模型端点（EndPoint）以外，`OpenAI Agents SDK`官方兼容了使用`LiteLLM`项目托管的模型服务接入。`LiteLLM` 是一个开源项目，于2023年7月27日开源。该项目集成了 100+ 主流模型，Github上开源地址：https://github.com/BerriAI/litellm

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202505061557327.png" width=60%></div>

&emsp;&emsp;其中该项目框架支持模型的完整列表可以在这里找到：https://docs.litellm.ai/docs/providers

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202505071740592.png" width=60%></div>

&emsp;&emsp;`OpenAI Agents SDK`通过实现兼容`LiteLLM`规范的模型提供类（ModelProvider），从而以支持通过该平台框架模型的接入。源码位置为： https://github.com/openai/openai-agents-python/blob/main/src/agents/extensions/models/litellm_provider.py

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202505071744085.png" width=60%></div>

&emsp;&emsp;因此，如果想要使用`LiteLLM`接入非`OpenAI`的`GPT`系列模型，我们首先需要安装`litellm`依赖项组，执行如下命令：

In [13]:
# ! pip install "openai-agents[litellm]"

&emsp;&emsp;安装完成后，直接导入`LitellmModel`类。                 

In [15]:
from agents.extensions.models.litellm_model import LitellmModel

&emsp;&emsp;接下来导入`DeepSeek`模型的配置参数：

In [14]:
# 读取DeepSeek API-KEY
DEEPSEEK_API_KEY = os.getenv("DEEPSEEK_API_KEY")
DEEPSEEK_BASE_URL = os.getenv("DEEPSEEK_BASE_URL")
DEEPSEEK_MODEL = os.getenv("DEEPSEEK_MODEL")

&emsp;&emsp;这里需要注意到是：`api_key`和`base_url`依然可以与使用`OpenAIChatCompletionsModel`传递，但是`model_name`，则需要根据`liteLLM`集成的模型名称来设置，而不再是`DeepSeek`官网模型名称。即：`LiteLLM`框架中的配置优先级最高。比如对于`DeepSeek`模型来说： https://docs.litellm.ai/docs/providers/deepseek

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202505071752392.png" width=60%></div>

&emsp;&emsp;因此，在实例化模型的时候，要添加`deepseek`前缀，而不能仅仅填写`deepseek-chat`。（其他模型也需要依照`litellm`框架的规范进行配置）

In [16]:
from agents import ModelSettings

agent = Agent(
    name="乐于助人的私人小助理",
    instructions="请使用中文回答用户的问题",
    model=LitellmModel(
        model="deepseek/deepseek-chat", 
        api_key=DEEPSEEK_API_KEY, 
        base_url=DEEPSEEK_BASE_URL),
    model_settings=ModelSettings(
        temperature=0.7,
        max_tokens=2048,
    )
)

result = await Runner.run(agent, "请你介绍一下你自己")
print(result.final_output)

你好！我是 **DeepSeek Chat**，由 **深度求索（DeepSeek）** 研发的一款智能 AI 助手。我可以帮助你解答各种问题，包括学习、工作、编程、生活百科、创意写作等。  

### **我的特点：**  
✅ **知识丰富**：我的知识截止到 **2024 年 7 月**，可以为你提供最新的信息（但无法实时更新）。  
✅ **超长上下文**：支持 **128K** 上下文，可以处理超长文档，适合阅读论文、分析代码等复杂任务。  
✅ **文件阅读**：支持 **PDF、Word、Excel、PPT、TXT** 等文件上传，并能从中提取关键信息。  
✅ **免费使用**：目前 **完全免费**，没有隐藏收费，你可以随时向我提问！  
✅ **多语言支持**：可以用 **中文、英文** 等多种语言交流，还能帮你翻译、润色文章等。  

### **我能帮你做什么？**  
📚 **学习辅导**：数学、物理、历史、编程等学科问题。  
💼 **工作助手**：写邮件、做 PPT、整理数据、优化简历。  
💡 **创意写作**：写小说、广告文案、诗歌、剧本等。  
📊 **代码编写**：Python、Java、C++ 等编程语言，调试优化代码。  
🌍 **生活百科**：旅行建议、健康小知识、美食推荐等。  

你可以随时向我提问，我会尽力提供最专业、最贴心的回答！😊 有什么我可以帮你的吗？


&emsp;&emsp;如果想要切换成推理模型 DeepSeek-R1，只需要修改模型名称即可。

In [17]:
agent = Agent(
    name="乐于助人的私人小助理",
    instructions="请使用中文回答用户的问题",
    model=LitellmModel(
        model="deepseek/deepseek-reasoner",  # 这里切换模型
        api_key=DEEPSEEK_API_KEY, 
        base_url=DEEPSEEK_BASE_URL),
    model_settings=ModelSettings(
        temperature=0.7,
        max_tokens=2048,
    )
)
result = await Runner.run(agent, "请你介绍一下你自己")
print(result.final_output)


你好！我是DeepSeek-R1，一个由中国的深度求索（DeepSeek）公司开发的智能助手。我擅长通过对话形式与用户互动，能够回答问题、提供建议、辅助学习、创作内容，还能处理数学计算、编程协助等多元化需求。我的知识库覆盖多个领域，但也会明确告知能力边界，对于不确定的信息会如实说明。

我的核心技术基于深度学习和大规模数据训练，通过算法理解用户意图并生成响应。需要特别说明的是，我和其他主流AI模型一样，不会存储用户对话记录，所有交流内容会在对话结束后自动清除，充分保护用户隐私。

如果你有任何问题或需要帮助，现在就可以直接告诉我哦~


&emsp;&emsp;以上就是`OpenAI Agents SDK`接入大模型服务的多种方法，当使用非`OpenAI`模型时，借助`LiteLLM`框架是最简单的一种接入方法。但根据官方的说明，目前`LiteLLM` 集成仍处于测试阶段，所以当接入的模型出现一些兼容性问题时，建议尝试使用`OpenAIChatCompletionsModel`类构建自定义接入流程。

&emsp;&emsp;熟悉了不同模型的接入方法后，接下来我们则需要重点了解`Runner`的内置实现原理及进阶的应用方法。其中`Agent`组件中未讲解的核心参数（非绿色标识），我们也将结合`Runner`的运行机制来进一步探讨其应用的价值。

<style>
.center 
{
  width: auto;
  display: table;
  margin-left: auto;
  margin-right: auto;
}
</style>

<p align="center"><font face="黑体" size=4>Agent 组件核心参数</font></p>
<div class="center">

| 属性名                | 类型                                                                                          | 描述                                                                                                   |
|---------------------|-----------------------------------------------------------------------------------------------|--------------------------------------------------------------------------------------------------------|
| <font color=green>**name**</font>             | `str`                                                                                         | 代理的名称。                                                                                             |
| <font color=green>**instructions**</font>      | `str` \| `Callable[[RunContextWrapper[TContext], Agent[TContext]], MaybeAwaitable[str]]` \| `None` | 代理的指令，用作“系统提示”。可以是字符串或动态生成指令的函数。                                           |
| `handoff_description` | `str` \| `None`                                                                              | 代理的描述，用于代理作为交接时，让 LLM 知道它的功能和何时调用它。                                         |
| `handoffs`          | list[Agent[Any] | Handoff[TContext]]                                                   | 代理可以委托的子代理列表。允许关注点分离和模块化。                                                       |
| <font color=green>**model**</font>              | `str` \| `Model` \| `None`                                                                   | 调用 LLM 时使用的模型实现。默认情况下，如果未设置，代理将使用 `openai_provider.DEFAULT_MODEL` 中配置的默认模型。 |
| <font color=green>**model_settings**</font>    | `ModelSettings`                                                                               | 配置模型特定的调优参数（例如温度、top_p）。                                                             |
| `tools`             | `list[Tool]`                                                                                 | 代理可以使用的工具列表。                                                                                 |
| `mcp_servers`       | `list[MCPServer]`                                                                             | 代理可以使用的模型上下文协议（MCP）服务器列表。                                                        |
| `mcp_config`        | `MCPConfig`                                                                                   | MCP 服务器的配置。                                                                                       |
| `input_guardrails`  | `list[InputGuardrail[TContext]]`                                                             | 在代理执行之前并行运行的检查列表，仅在代理是链中的第一个代理时运行。                                     |
| `output_guardrails` | `list[OutputGuardrail[TContext]]`                                                            | 在生成响应后对代理的最终输出运行的检查列表，仅在代理生成最终输出时运行。                                 |
| `output_type`       | type[Any] | AgentOutputSchemaBase | None                                                 | 输出对象的类型。如果未提供，输出将为 `str`。                                                             |
| `hooks`             | AgentHooks[TContext] | None                                                               | 接收代理生命周期事件回调的类。                                                                           |
| `tool_use_behavior` | Literal["run_llm_again", "stop_on_first_tool"] | StopAtTools | ToolsToFinalOutputFunction | 配置工具使用的处理方式。                                                                                 |
| `reset_tool_choice` | `bool`                                                                                       | 调用工具后是否将工具选择重置为默认值。默认为 `True`。确保代理不会进入工具使用的无限循环。                   |

## 2. Runner 核心组件详解

&emsp;&emsp;上一小节我们提到过：`OpenAI Agents SDK`框架中`Agent`组件的本质是初始化一系列的“静态属性”，真正构建“代理运行时”的组件是`Runner`组件。`Runner`方法其源码位置：https://github.com/openai/openai-agents-python/blob/main/src/agents/run.py ：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202505071818798.png" width=60%></div>     

&emsp;&emsp;在源码定义中可以清楚的看到，`Runner`组价一共提供了三个方法来运行代理，分别是：

- **Runner.run()** ：异步运行并返回最终响应结果。
- **Runner.run_sync()**： 同步运行，本质上是对异步 `run()` 方法的封装，从而可以在没有事件循环的情况下（例如在普通的 Python 脚本或某些环境中）以同步方式执行代理的逻辑。
- **Runner.run_streamed()**： ，异步运行并返回最终的响应结果。它以流模式调用大模型，并在接收到事件时将其进行流式的实时传输。

&emsp;&emsp;因此大家现在就应该能够理解，为什么我们在之前定义好`Agent`以后，直接调用`Runner.run()`便可以得到模型的最终响应结果。

In [18]:
from openai import AsyncOpenAI
import os
from dotenv import load_dotenv
from agents import OpenAIChatCompletionsModel, ModelSettings

# 加载环境变量
load_dotenv(override=True)


# 读取DeepSeek API-KEY
DEEPSEEK_API_KEY = os.getenv("DEEPSEEK_API_KEY")
DEEPSEEK_BASE_URL = os.getenv("DEEPSEEK_BASE_URL")
DEEPSEEK_MODEL = os.getenv("DEEPSEEK_MODEL")


# 定义模型实例
deepseek_client = AsyncOpenAI(
    base_url=DEEPSEEK_BASE_URL, 
    api_key=DEEPSEEK_API_KEY
    )

# 定义Agent配置
deepseek_agent = Agent(
    name="乐于助人的私人小助理",
    instructions="请使用中文回答用户的问题",
    model=OpenAIChatCompletionsModel(
        model=DEEPSEEK_MODEL,
        openai_client=deepseek_client,
    ),
    model_settings=ModelSettings(
        temperature=0.6,
        max_tokens=2048,
    )
)

# 通过 Runner 运行 Agent 并获得最终的模型响应
result = await Runner.run(agent, "请你介绍一下你自己")
print(result.final_output)

您好！我是DeepSeek-R1，一个由深度求索（DeepSeek）公司开发的智能助手，擅长通过思考解答复杂问题，提供多领域信息查询、语言翻译、创意灵感及生活建议等服务。我的核心能力基于Transformer架构的预训练模型，能流畅处理中文、英文及其他多种语言场景。

无论您需要日常闲聊、学习辅助、职场技巧还是科技知识，我都能用简洁易懂的方式与您互动。同时我会严格遵循隐私保护原则，注重对话安全性。若有任何具体需求，欢迎随时告诉我，我会尽力协助您：）


&emsp;&emsp;不论使用`Runner`的哪种方法运行代理，其能够接收的参数都如下所示：

<style>
.center 
{
  width: auto;
  display: table;
  margin-left: auto;
  margin-right: auto;
}
</style>

<p align="center"><font face="黑体" size=4>Runner 组件核心参数</font></p>
<div class="center">


| 属性名                     | 描述                                                                                                   |
|--------------------------|--------------------------------------------------------------------------------------------------------|
| <font color=red>**starting_agent**</font>        | 要运行的起始代理。                                                                                       |
|  <font color=red>**input**</font>                     | 代理的初始输入。可以传递一个用户消息的字符串，或一个输入项的列表。                                           |
| `context`                | 运行代理时使用的上下文。                                                                                 |
| `max_turns`             | 运行代理的最大回合数。回合定义为一次 AI 调用（包括可能发生的任何工具调用）。                                   |
| `hooks`                  | 接收各种生命周期事件回调的对象。                                                                         |
| `run_config`             | 整个代理运行的全局设置。                                                                                 |
| `previous_response_id`   | 上一个响应的 ID，如果使用 OpenAI 模型通过 Responses API，这允许你跳过传递上一个回合的输入。                   |

&emsp;&emsp;所以我们之前一直使用的最简实现形式，如果规范化定义则如下代码所示：

In [19]:
from agents import Runner

result = await Runner.run(
    starting_agent=deepseek_agent,
    input="你好，请你介绍一下你自己"
    )

print(result.final_output)

你好！我是DeepSeek Chat，由深度求索公司（DeepSeek）研发的智能AI助手。我可以帮助你解答各种问题，包括学习、工作、编程、生活百科、科技、娱乐等多个领域。  

### 我的特点：  
✅ **知识丰富**：我的知识截止到2024年7月，可以为你提供最新的科技、新闻、学术等信息。  
✅ **超长上下文**：支持**128K**上下文记忆，能处理超长文档，适合阅读论文、分析报告等。  
✅ **文件阅读**：可以解析**PDF、Word、Excel、PPT、TXT**等文件，帮助你提取关键信息。  
✅ **免费使用**：目前不收费，你可以随时向我提问！  
✅ **多语言支持**：可以用中文、英文等多种语言交流。  

无论是查资料、写作润色、代码调试，还是日常闲聊，我都可以帮到你！😊 你有什么想问的呢？


&emsp;&emsp;除此以外，一种更加规范的传入输入文本的写法如下：

In [28]:
from agents import Runner

result = await Runner.run(
    starting_agent=deepseek_agent,
    input=[{
        "role": "user",
        "content": "你好，请你介绍一下你自己"
    }]
)

print(result.final_output)

你好！我是 **DeepSeek Chat**，由深度求索公司（DeepSeek）研发的一款智能AI助手。我可以帮助你解答各种问题，包括学习、工作、编程、生活百科、创意写作等。  

### **我的特点：**  
✅ **知识丰富**：我的知识截止到**2024年7月**，可以回答各种领域的问题。  
✅ **超长上下文**：支持**128K**上下文，能记住更长的对话内容。  
✅ **文件阅读**：可以解析 **PDF、Word、Excel、PPT、TXT** 等文件，帮助你提取关键信息。  
✅ **免费使用**：目前**完全免费**，没有隐藏收费！  
✅ **中文优化**：对中文理解和生成特别优化，适合中文用户。  

### **我能帮你做什么？**  
📚 **学习辅导**：数学、物理、历史、编程等学科问题。  
💼 **工作助手**：写邮件、做PPT、数据分析、简历优化等。  
💡 **创意写作**：写小说、广告文案、诗歌、剧本等。  
📊 **代码编写**：Python、C++、Java、SQL等编程语言支持。  
🌍 **生活百科**：旅行建议、健康知识、美食推荐等。  

你可以随时向我提问，我会尽力提供**专业、准确、友好**的回答！😊 有什么我可以帮你的吗？


## 3. 上下文管理

&emsp;&emsp;`Runner`运行过程就是在实际处理模型响应、工具调用、MCP服务器服务器运行等一系列自主行为从而自动化的完成特定的复杂任务。而`Agent`在执行某一次运行时，一般都需要关注以下两个方面：
1. `Agent` 运行时可用的数据：这指的是工具函数/MCP服务器、各种回调运行时需要的数据和依赖项。比如上一个工具的输出结果会作为下一个工具的输入。
2. `Model` 可用的数据：这指的是大模型在生成最终响应时能够看到的数据。比如历史输入、历史输出、历史工具调用、历史工具调用结果等。


&emsp;&emsp;这两种不同的数据生成及传递，在`OpenAI Agents SDK`框架中称之为统称为`上下文`（Context）。


&emsp;&emsp;因此，在`Runner`的三种运行方法中（`run`、`run_async`、`run_streamed`），均设置了一个`context`参数，以用来进行上下文数据的传输。其源码位置：https://github.com/openai/openai-agents-python/blob/main/src/agents/run.py

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202505081053833.png" width=60%></div>

&emsp;&emsp;`Context Management` 通过`RunContextWrapper`类及其中的`context` 属性来表示。其构建流程如下：

1. 使用数据类（data class）或 Pydantic 对象创建上下文
2. 将该对象传递给 `Runner` 中的运行方法，比如 `Runner.run()` 、`Runner.run_sync()` 或者 `Runner.run_streamed()`
3. 在 `Runner` 的运行过程中，`RunContextWrapper` 会自动将上下文数据传递给 `Agent` 和 `Model` 对象
4. 在 `Agent` 和 `Model` 对象中，上下文数据可以通过 `context` 属性访问

&emsp;&emsp;接下来我们可以通过一些实际的示例来了解`Context Management`的实现，并明确`Agent`和`Model`在运行过程中如何使用上下文数据以及适用的开发模式。


### 3.1 Agent Local Context

&emsp;&emsp;`Agent` 运行状态下所用的数据在`OpenAI Agents SDK`框架中称之为`Local Context`。它的形式是一个`Python`对象，在整个`Agent`运行期间持续存在。它的主要目的是在代码执行层面提供数据共享和状态管理。（其实就类似于`LangGraph`框架中的`State`）。

&emsp;&emsp;`Agent`能够执行复杂任务的核心是可以执行函数调用，而函数调用的两种参数来源主要有两种：

1. **大模型解析的显示参数：** 通过 JsonSchema 定义，由大模型理解用户意图后提供；即工具函数的注释描述部分；
2. **人工构建的隐式参数：** 由开发人员预先设置，对大模型不可见，即需要借助`Local Context`来传递；

&emsp;&emsp;`Local Context` 的核心价值之一在于可以将特定的数据和业务逻辑与大模型执行工具的解析过程分离。比如下面的这个场景：
- 隐式参数（上下文）：`wrapper.context` 中包含的用户基本信息，如ID、姓名等；
- 显式参数：`info_type` 和 `format_type`，由大模型从用户自然语言中解析；

In [20]:
from dataclasses import dataclass
from typing import Optional
from agents import RunContextWrapper, function_tool


# 通过数据类来定义上下文类
@dataclass
class UserInfo:  
    name: str
    uid: int
    birthday: str = "1995-01-23"  # 添加用户生日信息
    location: str = "北京"        # 添加用户位置信息

# 定义工具函数
@function_tool
async def fetch_user_info(
    wrapper: RunContextWrapper[UserInfo],     # 隐式参数 - 上下文
    info_type: str,                           # 显式参数 - 由大模型解析用户想查询的信息类型
    format_type: Optional[str] = None         # 显式参数 - 可选的格式化方式
) -> str:  
    """
    获取用户的详细信息
    
    参数:
    - info_type: 要获取的信息类型，如"年龄"、"生日"、"位置"等
    - format_type: 可选的信息格式，如"简洁"、"详细"
    """
    # 从隐式上下文参数获取用户基本信息
    user_name = wrapper.context.name
    user_id = wrapper.context.uid
    
    # 根据显式参数决定返回什么信息
    if info_type.lower() == "年龄":
        info = f"30 岁"
    elif info_type.lower() == "生日":
        info = wrapper.context.birthday
    elif info_type.lower() == "位置":
        info = wrapper.context.location
    else:
        info = "未知信息类型"
    
    # 根据显式参数决定返回格式
    if format_type and format_type.lower() == "详细":
        return f"用户详细信息 - ID: {user_id}, 姓名: {user_name}, {info_type}: {info}"
    else:
        return f"用户 {user_name} 的{info_type}是 {info}"


&emsp;&emsp;在构建`Agent`的时候，使用`tools`参数来传递工具函数。

<style>
.center 
{
  width: auto;
  display: table;
  margin-left: auto;
  margin-right: auto;
}
</style>

<p align="center"><font face="黑体" size=4>Agent 组件核心参数</font></p>
<div class="center">

| 属性名                | 类型                                                                                          | 描述                                                                                                   |
|---------------------|-----------------------------------------------------------------------------------------------|--------------------------------------------------------------------------------------------------------|
| <font color=green>**name**</font>             | `str`                                                                                         | 代理的名称。                                                                                             |
| <font color=green>**instructions**</font>      | `str` \| `Callable[[RunContextWrapper[TContext], Agent[TContext]], MaybeAwaitable[str]]` \| `None` | 代理的指令，用作“系统提示”。可以是字符串或动态生成指令的函数。                                           |
| `handoff_description` | `str` \| `None`                                                                              | 代理的描述，用于代理作为交接时，让 LLM 知道它的功能和何时调用它。                                         |
| `handoffs`          | list[Agent[Any] | Handoff[TContext]]                                                   | 代理可以委托的子代理列表。允许关注点分离和模块化。                                                       |
| <font color=green>**model**</font>              | `str` \| `Model` \| `None`                                                                   | 调用 LLM 时使用的模型实现。默认情况下，如果未设置，代理将使用 `openai_provider.DEFAULT_MODEL` 中配置的默认模型。 |
| <font color=green>**model_settings**</font>    | `ModelSettings`                                                                               | 配置模型特定的调优参数（例如温度、top_p）。                                                             |
| <font color=red>**tools**</font>             | `list[Tool]`                                                                                 | 代理可以使用的工具列表。                                                                                 |
| `mcp_servers`       | `list[MCPServer]`                                                                             | 代理可以使用的模型上下文协议（MCP）服务器列表。                                                        |
| `mcp_config`        | `MCPConfig`                                                                                   | MCP 服务器的配置。                                                                                       |
| `input_guardrails`  | `list[InputGuardrail[TContext]]`                                                             | 在代理执行之前并行运行的检查列表，仅在代理是链中的第一个代理时运行。                                     |
| `output_guardrails` | `list[OutputGuardrail[TContext]]`                                                            | 在生成响应后对代理的最终输出运行的检查列表，仅在代理生成最终输出时运行。                                 |
| `output_type`       | type[Any] | AgentOutputSchemaBase | None                                                 | 输出对象的类型。如果未提供，输出将为 `str`。                                                             |
| `hooks`             | AgentHooks[TContext] | None                                                               | 接收代理生命周期事件回调的类。                                                                           |
| `tool_use_behavior` | Literal["run_llm_again", "stop_on_first_tool"] | StopAtTools | ToolsToFinalOutputFunction | 配置工具使用的处理方式。                                                                                 |
| `reset_tool_choice` | `bool`                                                                                       | 调用工具后是否将工具选择重置为默认值。默认为 `True`。确保代理不会进入工具使用的无限循环。                   |

In [21]:
import os
from agents import Agent, Runner, OpenAIChatCompletionsModel, ModelSettings
from openai import AsyncOpenAI

# 读取DeepSeek API-KEY
DEEPSEEK_API_KEY = os.getenv("DEEPSEEK_API_KEY")
DEEPSEEK_BASE_URL = os.getenv("DEEPSEEK_BASE_URL")
DEEPSEEK_MODEL = os.getenv("DEEPSEEK_MODEL")

deepseek_client = AsyncOpenAI(base_url=DEEPSEEK_BASE_URL, 
                     api_key=DEEPSEEK_API_KEY)

# 创建代理
userinfo_service_agent = Agent[UserInfo](
    name="客户服务助手",
    instructions="你是一个帮助查询用户信息的助手",
    model=OpenAIChatCompletionsModel(
        model=DEEPSEEK_MODEL,
        openai_client=deepseek_client,
    ),
    model_settings=ModelSettings(
        temperature=0.6,
        max_tokens=2048,
    ),
    tools=[fetch_user_info]
)

&emsp;&emsp;构建用户信息上下文对象：

In [22]:
user_info = UserInfo(name="木羽", uid=123456)

&emsp;&emsp;然后，在构建`Runner`运行实例的时候，通过`context`参数来传递`user_info`上下文类对象。

<style>
.center 
{
  width: auto;
  display: table;
  margin-left: auto;
  margin-right: auto;
}
</style>

<p align="center"><font face="黑体" size=4>Runner 组件核心参数</font></p>
<div class="center">


| 属性名                     | 描述                                                                                                   |
|--------------------------|--------------------------------------------------------------------------------------------------------|
| <font color=green>**starting_agent**</font>        | 要运行的起始代理。                                                                                       |
|  <font color=green>**input**</font>                     | 代理的初始输入。可以传递一个用户消息的字符串，或一个输入项的列表。                                           |
| <font color=red>context<font color=red>                | 运行代理时使用的上下文。                                                                                 |
| `max_turns`             | 运行代理的最大回合数。回合定义为一次 AI 调用（包括可能发生的任何工具调用）。                                   |
| `hooks`                  | 接收各种生命周期事件回调的对象。                                                                         |
| `run_config`             | 整个代理运行的全局设置。                                                                                 |
| `previous_response_id`   | 上一个响应的 ID，如果使用 OpenAI 模型通过 Responses API，这允许你跳过传递上一个回合的输入。                   |

In [23]:
# 示例1: 查询年龄（基本用法）
result1 = await Runner.run(  
    starting_agent=userinfo_service_agent,
    input="请告诉我木羽的年龄",
    context=user_info,  # 通过context参数来传递上下文类对象
)
print("===== 查询年龄 =====")
print(result1.final_output)

===== 查询年龄 =====
木羽的年龄是30岁。


&emsp;&emsp;查看大模型解析的参数：

In [24]:
result1.new_items[0].raw_item

ResponseFunctionToolCall(arguments='{"info_type":"年龄","format_type":null}', call_id='call_0_17f18b6e-4115-4770-bde2-cd07fb573410', name='fetch_user_info', type='function_call', id='__fake_id__', status=None)

&emsp;&emsp;通过回复结果可以看到，大模型解析的参数仅为`info_type`和`format_type`，而`wrapper.context`中包含的用户基本信息，如ID、姓名等，则通过`context`参数来传递。

In [25]:
result2 = await Runner.run(  
    starting_agent=userinfo_service_agent,
    input="请详细告诉我木羽住在哪里",
    context=user_info,
)

print("\n===== 查询位置（详细格式） =====")
print(result2.final_output)


===== 查询位置（详细格式） =====
木羽目前住在北京。


&emsp;&emsp;再次查看大模型解析的参数：


In [26]:
result2.new_items[0].raw_item

ResponseFunctionToolCall(arguments='{"info_type":"位置","format_type":"详细"}', call_id='call_0_bd6a0c32-dac8-4486-b29f-105cae01306c', name='fetch_user_info', type='function_call', id='__fake_id__', status=None)

&emsp;&emsp;同样符合预期的解析逻辑。以上这个示例可以清晰地展示上下文和大模型解析参数如何协同工作，使函数工具既能访问用户的基本信息（通过上下文），又能响应用户的具体请求（通过显式参数）。这种分离形式在实际应用中非常有用，同时对模型不可见的实现思路，一方面可以避免大模型解析错误，或者解析参数过多，导致上下文数据过大，同时针对隐私数据也能起到非常强的保护作用，避免发生数据泄露。


&emsp;&emsp;`Local Context`的另一个重要应用场景是多代理协作中的状态共享。


&emsp;&emsp;下面的这个案例重点展示了如何利用共享上下文（Local Context）实现多智能体之间的协作，通过典型的金融服务`WorkFlow`：客户信息查询 → 交易历史查询 → 产品推荐，借助上下文共享实现无缝衔接，从而构建一个完整的金融服务流程。系统包含两个专业智能体：

- 客户信息助手：负责查询客户基本信息和交易历史
- 产品推荐助手：基于客户信息推荐合适的金融产品

In [27]:
# ! pip install nest_asyncio
import os
from dotenv import load_dotenv
load_dotenv(override=True)

import nest_asyncio
nest_asyncio.apply()

- **阶段一：上下文设计与初始化**

&emsp;&emsp;如下代码所示，多个智能体共享的上下文类`MySQLContext`包含共享的数据库连接池，跟踪当前正在服务的客户，以及查询结果缓存。

In [28]:
from dataclasses import dataclass, field
from typing import List, Dict, Any, Optional
import pymysql
from pymysql.cursors import DictCursor
from datetime import datetime, date
from dbutils.pooled_db import PooledDB
from faker import Faker
import json
from agents import Agent, RunContextWrapper, Runner, function_tool, handoff

# 创建Faker实例用于生成模拟数据
fake = Faker('zh_CN')

@dataclass
class MySQLContext:
    """管理MySQL连接和查询状态的上下文类"""
    connection_pool: PooledDB  # 共享数据库连接池
    customer_id: str  # 当前正在服务的客户ID
    query_results: Dict[str, Any] = field(default_factory=dict) # 用于存储查询结果
    current_step: str = "客户信息查询" # 当前步骤
    workflow_complete: bool = False # 是否完成工作流
    
    def get_connection(self):
        """从连接池获取一个连接"""
        return self.connection_pool.connection()

&emsp;&emsp;然后，创建连接池。注意：这里需要先启动`Mysql`服务，然后替换成实际使用的`Mysql`连接信息。

In [29]:
# 创建连接池 - 使用您提供的dbutils和pymysql实现
pool = PooledDB(
    creator=pymysql,       # 使用pymysql模块
    maxconnections=5,      # 连接池允许的最大连接数
    host="localhost",
    user="root",
    password="Snowball2019",  # 使用您提供的密码
    port=3306,
    database="financial_service",
    charset='utf8mb4',
    connect_timeout=10     # 连接超时时间
)
    

&emsp;&emsp;然后，这里我们使用`Faker`生成一些模拟数据。已经随课件提供的脚本文件名为`faker_data.py`。只需要注意修改数据库连接信息，然后运行脚本生成数据即可。


<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202505081657553.png" width=60%></div>     

&emsp;&emsp;执行完成后，则可以在本地的`Mysql`数据库中看到生成的金融模拟数据。


<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202505081659369.png" width=60%></div>     
&emsp;&emsp;


- **阶段二：创建客户信息助手（第一个智能体）**

&emsp;&emsp;客户信息助手智能体负责查询客户的基本信息，我们给该智能体涉及两个工具函数。第一个工具函数是`query_customer_info`，用于查询客户的基本信息，该工具函数包含显式和隐式参数。

- 显式参数：`include_sensitive_info` 和 `format_type`，由大模型解析用户想查询的信息类型；
- 隐式参数：`context`，通过上下文类`MySQLContext`来传递，包含数据库连接池、当前客户ID、查询结果缓存、当前步骤等。

&emsp;&emsp;这个函数负责查询客户的基本个人信息，包括姓名、账户类型、余额、联系方式等。其中通过`include_sensitive_info`参数来控制是否包含敏感信息，通过`format_type`参数来控制输出格式。

In [30]:
@function_tool
async def query_customer_info(
    context: RunContextWrapper[MySQLContext],           # 隐式参数 - 上下文
    include_sensitive_info: Optional[bool] = False,     # 显式参数 - 是否包含敏感信息
    format_type: Optional[str] = "standard"             # 显式参数 - 输出格式
) -> str:
    """
    查询客户的基本信息
    
    参数:
    - include_sensitive_info: 是否包含敏感信息如电话和邮箱
    - format_type: 输出格式类型，可选"standard"、"brief"或"detailed"   
    """
    connection = None
    cursor = None
    try:
        # 从上下文获取隐式参数 
        connection = context.context.get_connection()   # 从上下文获取数据库连接
        cursor = connection.cursor(DictCursor)          # 创建游标
        customer_id = context.context.customer_id      # 从上下文获取当前客户ID
        
        # 查询客户基本信息
        cursor.execute("""
        SELECT id, name, account_type, balance, phone, email, address, registration_date
        FROM customers
        WHERE id = %s
        """, (customer_id,))
        
        customer = cursor.fetchone()  # 获取查询结果
        if not customer:
            return f"未找到ID为{customer_id}的客户信息"
        
        # 将日期转换为字符串
        if 'registration_date' in customer and isinstance(customer['registration_date'], (datetime, date)):
            customer['registration_date'] = customer['registration_date'].isoformat()
            
        # 存储查询结果到上下文
        context.context.query_results["customer_info"] = customer  # 将查询结果存储到上下文
        context.context.current_step = "交易历史查询"              # 更新当前步骤
        
        # 根据显式参数处理敏感信息
        if not include_sensitive_info:
            phone = "***********" if 'phone' in customer else "未提供"  # 如果包含敏感信息，则将电话和邮箱隐藏
            email = "***********" if 'email' in customer else "未提供"  # 如果包含敏感信息，则将电话和邮箱隐藏
        else:
            phone = customer.get('phone', "未提供")  # 如果不需要隐藏敏感信息，则直接返回电话和邮箱
            email = customer.get('email', "未提供")  # 如果不需要隐藏敏感信息，则直接返回电话和邮箱
        
        # 根据显式参数格式化输出
        if format_type.lower() == "brief":
            # 简要格式
            info = (
                f"客户 {customer['name']} ({customer['account_type']})\n"
                f"账户余额: {customer['balance']} 元"
            )
        elif format_type.lower() == "detailed":
            # 详细格式
            info = (
                f"客户详细信息（ID: {customer_id}）:\n"
                f"- 姓名: {customer['name']}\n"
                f"- 账户类型: {customer['account_type']}\n"
                f"- 账户余额: {customer['balance']} 元\n"
                f"- 电话: {phone}\n"
                f"- 邮箱: {email}\n"
                f"- 地址: {customer['address']}\n"
                f"- 注册日期: {customer['registration_date']}\n"
                f"- 客户价值评估: {'高' if float(customer['balance']) > 500000 else '中' if float(customer['balance']) > 100000 else '一般'}"
            )
        else:
            # 标准格式
            info = (
                f"客户信息:\n"
                f"- 姓名: {customer['name']}\n"
                f"- 账户类型: {customer['account_type']}\n"
                f"- 账户余额: {customer['balance']} 元\n"
                f"- 电话: {phone}\n"
                f"- 邮箱: {email}\n"
                f"- 地址: {customer['address']}\n"
                f"- 注册日期: {customer['registration_date']}"
            )
        
        return info
    
    except Exception as e:
        return f"查询客户信息时发生错误: {str(e)}"
    
    finally:
        if cursor:
            cursor.close()
        if connection:
            connection.close()

&emsp;&emsp;第二个工具函数是`query_transaction_history`，用于查询客户的基本交易历史，通过上下文获取数据库连接和客户ID，并利用上一步已查询的信息，构建查询SQL。`transaction_type`由大模型解析用户想过滤的交易类型（存款、取款等），而`min_amount`则是由模型解析用户指定的最小交易金额。

In [31]:
@function_tool
async def query_transaction_history(
    context: RunContextWrapper[MySQLContext],           # 隐式参数 - 上下文
    days: Optional[int] = 180,                           # 显式参数 - 查询天数
    transaction_type: Optional[str] = None,             # 显式参数 - 交易类型过滤
    min_amount: Optional[float] = None                  # 显式参数 - 最小金额过滤
) -> str:
    """
    查询客户的交易历史
    
    参数:
    - days: 查询最近几天的交易，默认30天
    - transaction_type: 交易类型过滤，如"存款"、"取款"、"购买"、"赎回" 
    - min_amount: 最小交易金额过滤
    """
    connection = None
    cursor = None
    try:
        # 从上下文获取隐式参数
        connection = context.context.get_connection()   # 从上下文获取数据库连接
        cursor = connection.cursor(DictCursor)          # 创建游标
        customer_id = context.context.customer_id      # 从上下文获取当前客户ID
        
        # 构建查询SQL - 使用显式参数
        base_query = """
        SELECT t.id, t.amount, t.transaction_date, t.transaction_type, t.status, p.name as product_name
        FROM transactions t
        JOIN products p ON t.product_id = p.id
        WHERE t.customer_id = %s
        """
        
        query_params = [customer_id]
        
        # 添加日期过滤条件
        if days:
            base_query += " AND t.transaction_date >= DATE_SUB(NOW(), INTERVAL %s DAY)"
            query_params.append(days)
        
        # 添加交易类型过滤条件
        if transaction_type:
            base_query += " AND t.transaction_type = %s"
            query_params.append(transaction_type)
        
        # 添加最小金额过滤条件
        if min_amount:
            base_query += " AND t.amount >= %s"
            query_params.append(min_amount)
        
        base_query += " ORDER BY t.transaction_date DESC LIMIT 10"
        
        # 执行查询
        cursor.execute(base_query, tuple(query_params))
        transactions = cursor.fetchall()
        
        # 转换日期为字符串
        for tx in transactions:
            if 'transaction_date' in tx and isinstance(tx['transaction_date'], (datetime, date)):
                tx['transaction_date'] = tx['transaction_date'].isoformat()
        
        # 存储查询结果到上下文
        context.context.query_results["transaction_history"] = transactions  # 将查询结果存储到上下文
        context.context.current_step = "产品推荐"                              # 更新当前步骤
        
        # 构建筛选条件描述
        filter_desc = []
        if days:
            filter_desc.append(f"最近{days}天")
        if transaction_type:
            filter_desc.append(f"类型为{transaction_type}")
        if min_amount:
            filter_desc.append(f"金额≥{min_amount}元")
        
        filter_text = "（筛选：" + "、".join(filter_desc) + "）" if filter_desc else ""
        
        if not transactions:
            return f"该客户{filter_text}没有交易记录"
        
        # 格式化输出交易历史
        history = f"最近交易记录{filter_text}:\n"
        for i, tx in enumerate(transactions, 1):
            history += (
                f"{i}. 产品: {tx['product_name']}\n"
                f"   金额: {tx['amount']} 元\n"
                f"   类型: {tx['transaction_type']}\n"
                f"   日期: {tx['transaction_date']}\n"
                f"   状态: {tx['status']}\n"
            )
        
        # 添加总结信息
        total_amount = sum(float(tx['amount']) for tx in transactions)
        history += f"\n共 {len(transactions)} 笔交易，总金额: {total_amount:.2f} 元"
        
        return history
    
    except Exception as e:
        return f"查询交易历史时发生错误: {str(e)}"
    
    finally:
        if cursor:
            cursor.close()
        if connection:
            connection.close()

&emsp;&emsp;定义好工具函数后，通过`tools`参数可以传入多个外部工具，`OpenAI Agents SDK`会自动将工具函数注册到代理中，同时支持多工具的并行调用。

In [33]:
# 创建代理
info_agent = Agent[MySQLContext](
    name="客户信息助手",
    instructions="请使用中文回答用户的问题",
    model=OpenAIChatCompletionsModel(
        model=DEEPSEEK_MODEL,
        openai_client=deepseek_client,
    ),
    model_settings=ModelSettings(
        temperature=0.6,
        max_tokens=2048,
    ),
    tools=[query_customer_info, query_transaction_history] # 通过 tools 参数接收工具函数
)

&emsp;&emsp;定义好代理后，通过`Runner`类可以启动工作流。首先我们来测试单个工具函数的解析和执行情况。注意，因为构建了上下文，所以并不需要在输入的问题中传递具体的用户ID等信息。

In [34]:
# 选择一个客户ID
# 从某个界面选择客户的过程
mysql_ctx = MySQLContext(
    connection_pool=pool,
    customer_id="CUST000100",  # 示例客户ID
)


# 第一步：客户信息查询
result1 = await Runner.run(
    starting_agent=info_agent,
    input="查询下客户的基本信息",
    context=mysql_ctx  # 传递上下文
)
print("\n===== 客户信息查询 =====")
print(result1.final_output)


===== 客户信息查询 =====
客户的简要信息如下：

- **姓名**: 何丹  
- **账户类型**: 普通  
- **账户余额**: 42,733.00 元  
- **地址**: 四川省刚市华龙阜新街D座 562358  
- **注册日期**: 2022-03-01  

如需更多详细信息，请告知。


&emsp;&emsp;通过`Runner`类可以获取到代理的输出结果，包括`final_output`和`new_items`。`final_output`是代理的最终输出结果，`new_items`是代理的输出结果中的新信息。在 `OpenAI Agents SDK` 中，`items` 用于定义和管理代理的行为。每个 `item` 代表代理在执行任务时生成的一个具体输出或操作，包括了行为定义和状态管理。如下所示：

<style>
.center 
{
  width: auto;
  display: table;
  margin-left: auto;
  margin-right: auto;
}
</style>

<p align="center"><font face="黑体" size=4>OpenAI Agents SDK Items</font></p>
<div class="center">

| 类型                          | 描述                                                                                     |
|-------------------------------|------------------------------------------------------------------------------------------|
| `ResponseFunctionToolCall`    | 表示一个工具调用的响应，包括调用的参数和状态。                                          |
| `ResponseOutputMessage`       | 表示来自 LLM 的消息输出，包含消息内容和角色信息。                                       |
| `ResponseOutputText`          | 表示消息中的文本内容，包含文本和相关注释。                                              |
| `ResponseOutputRefusal`       | 表示拒绝响应，包含拒绝的原因。                                                           |
| `ResponseStreamEvent`         | 表示流式响应事件，通常用于实时数据流的处理。                                            |
| `ResponseInputItemParam`      | 表示输入项的参数，通常用于传递给模型的输入数据。                                        |
| `ResponseOutputItem`          | 表示输出项，通常用于模型的返回结果。                                                    |
| `ResponseReasoningItem`       | 表示推理项，包含模型的推理过程和相关信息。                                              |
| `ResponseComputerToolCall`    | 表示自动执行计算机任务的响应，包含计算的结果和状态。                                          |
| `ResponseFileSearchToolCall`  | 表示文件搜索工具调用的响应，包含搜索结果和状态。                                        |
| `ResponseFunctionWebSearch`   | 表示网络搜索工具调用的响应，包含搜索结果和状态。                                        |


In [35]:
len(result1.new_items)

3

In [36]:
result1.new_items[0].raw_item

ResponseFunctionToolCall(arguments='{"include_sensitive_info":false,"format_type":"standard"}', call_id='call_0_f072cc97-750d-4956-bdc2-641116be9ef4', name='query_customer_info', type='function_call', id='__fake_id__', status=None)

In [37]:
result1.new_items[1].raw_item

{'call_id': 'call_0_f072cc97-750d-4956-bdc2-641116be9ef4',
 'output': '客户信息:\n- 姓名: 何丹\n- 账户类型: 普通\n- 账户余额: 42733.00 元\n- 电话: ***********\n- 邮箱: ***********\n- 地址: 四川省刚市华龙阜新街D座 562358\n- 注册日期: 2022-03-01',
 'type': 'function_call_output'}

In [38]:
result1.new_items[2].raw_item

ResponseOutputMessage(id='__fake_id__', content=[ResponseOutputText(annotations=[], text='客户的简要信息如下：\n\n- **姓名**: 何丹  \n- **账户类型**: 普通  \n- **账户余额**: 42,733.00 元  \n- **地址**: 四川省刚市华龙阜新街D座 562358  \n- **注册日期**: 2022-03-01  \n\n如需更多详细信息，请告知。', type='output_text')], role='assistant', status='completed', type='message')

&emsp;&emsp;同样，我们也可以测试交易历史查询工具函数的执行有效性。

In [39]:
# 第二步：交易历史查询
result2 = await Runner.run(
    starting_agent=info_agent,
    input="请查询客户近一年的存款记录",
    context=mysql_ctx
)
print("\n===== 交易历史查询 =====")
print(result2.final_output)


===== 交易历史查询 =====
客户近一年的存款记录如下：

1. **产品**: 定期存款-3个月  
   **金额**: 542,800.00 元  
   **类型**: 存款  
   **日期**: 2024年8月23日  
   **状态**: 已完成  

总计：1笔存款，总金额为542,800.00元。


In [40]:
len(result2.new_items)

3

In [41]:
result2.new_items[0].raw_item

ResponseFunctionToolCall(arguments='{"days":365,"transaction_type":"存款"}', call_id='call_0_9f770d4d-348c-4a5a-932c-dfad2746b541', name='query_transaction_history', type='function_call', id='__fake_id__', status=None)

&emsp;&emsp;除此以外，当输入的指令中包含多个工具调用时，`OpenAI Agents SDK` 会自动将多个工具调用组合成一个完整的任务，并按照工具调用的顺序依次执行。

In [42]:
# 同时查询客户的基本信息和交易历史
result3 = await Runner.run(
    starting_agent=info_agent,
    input="查一下客户的基本信息，以及近一年有没有赎回记录",
    context=mysql_ctx
)
print("\n===== 综合查询 =====")
print(result3.final_output)


===== 综合查询 =====
### 客户基本信息
- **姓名**: 何丹
- **账户类型**: 普通
- **账户余额**: 42,733.00 元
- **地址**: 四川省刚市华龙阜新街D座 562358
- **注册日期**: 2022-03-01

### 近一年赎回记录
- **交易记录**:
  1. **产品**: 债券基金  
     **金额**: 4,800.00 元  
     **类型**: 赎回  
     **日期**: 2024-07-27  
     **状态**: 已取消  
- **总计**: 1 笔交易，总金额 4,800.00 元  

如有其他需求，请随时告知！


In [43]:
len(result3.new_items)

5

In [44]:
result3.new_items[0].raw_item

ResponseFunctionToolCall(arguments='{"include_sensitive_info": false, "format_type": "standard"}', call_id='call_0_4cc9c7af-c172-467b-9917-beaf00da2a11', name='query_customer_info', type='function_call', id='__fake_id__', status=None)

In [45]:
result3.new_items[1].raw_item

ResponseFunctionToolCall(arguments='{"days": 365, "transaction_type": "赎回"}', call_id='call_1_72a4534b-3c47-4495-8da4-a4d5ee81330f', name='query_transaction_history', type='function_call', id='__fake_id__', status=None)

In [46]:
result3.new_items[2].raw_item

{'call_id': 'call_0_4cc9c7af-c172-467b-9917-beaf00da2a11',
 'output': '客户信息:\n- 姓名: 何丹\n- 账户类型: 普通\n- 账户余额: 42733.00 元\n- 电话: ***********\n- 邮箱: ***********\n- 地址: 四川省刚市华龙阜新街D座 562358\n- 注册日期: 2022-03-01',
 'type': 'function_call_output'}

In [47]:
result3.new_items[3].raw_item

{'call_id': 'call_1_72a4534b-3c47-4495-8da4-a4d5ee81330f',
 'output': '最近交易记录（筛选：最近365天、类型为赎回）:\n1. 产品: 债券基金\n   金额: 4800.00 元\n   类型: 赎回\n   日期: 2024-07-27T06:52:47\n   状态: 已取消\n\n共 1 笔交易，总金额: 4800.00 元',
 'type': 'function_call_output'}

In [48]:
result3.new_items[4].raw_item

ResponseOutputMessage(id='__fake_id__', content=[ResponseOutputText(annotations=[], text='### 客户基本信息\n- **姓名**: 何丹\n- **账户类型**: 普通\n- **账户余额**: 42,733.00 元\n- **地址**: 四川省刚市华龙阜新街D座 562358\n- **注册日期**: 2022-03-01\n\n### 近一年赎回记录\n- **交易记录**:\n  1. **产品**: 债券基金  \n     **金额**: 4,800.00 元  \n     **类型**: 赎回  \n     **日期**: 2024-07-27  \n     **状态**: 已取消  \n- **总计**: 1 笔交易，总金额 4,800.00 元  \n\n如有其他需求，请随时告知！', type='output_text')], role='assistant', status='completed', type='message')

- **阶段三： 构建产品推荐代理（第二个智能体）**

&emsp;&emsp;产品推荐助手智能体负责基于客户信息和交易历史推荐合适的金融产品，所以它是基于第一个智能体`info_agent`的下游代理。同样我们先定义一个工具函数`recommend_products`，用于基于客户信息和交易历史推荐产品。同样包含隐式和显式参数，其中：

- 隐式参数：通过上下文获取前两步已收集的客户信息和交易历史；
- 显式参数：`product_category`是由模型解析用户感兴趣的产品类别（储蓄、理财等），`risk_level`是由模型解析用户可接受的风险等级（低、中、高），`max_results`是由模型解析用户希望返回的产品数量；

In [49]:
@function_tool
async def recommend_products(
    context: RunContextWrapper[MySQLContext],           # 隐式参数 - 上下文
    product_category: Optional[str] = None,             # 显式参数 - 产品类别
    risk_level: Optional[str] = None,                   # 显式参数 - 风险等级
    max_results: Optional[int] = 6                      # 显式参数 - 最大结果数量
) -> str:
    """
    基于客户信息和交易历史推荐产品
    
    参数:
    - product_category: 产品类别过滤，如"储蓄"、"理财"、"保险"、"贷款"
    - risk_level: 风险等级过滤，如"低"、"中"、"高"
    - max_results: 最多返回几个产品推荐，默认6个
    """
    connection = None
    cursor = None
    try:
        # 从上下文获取隐式参数
        connection = context.context.get_connection() # 从上下文获取数据库连接
        cursor = connection.cursor(DictCursor) # 创建游标
        
        # 从上下文获取前面步骤查询的客户信息
        customer_info = context.context.query_results.get("customer_info", {}) # 从上下文获取客户信息
        if not customer_info:
            return "无法推荐产品，客户信息不可用"
        
        account_type = customer_info.get('account_type')
        balance = float(customer_info.get('balance', 0))
        
        # 构建基础查询
        base_query = """
        SELECT id, name, category, min_amount, interest_rate, term_months, risk_level
        FROM products
        WHERE min_amount <= %s
        """
        
        query_params = [balance]
        
        # 根据显式参数添加条件
        if product_category:
            base_query += " AND category = %s"
            query_params.append(product_category)
        
        if risk_level:
            base_query += " AND risk_level = %s"
            query_params.append(risk_level)
        
        # 根据用户类型决定是否包含VIP产品
        if account_type in ('VIP', '白金'):
            # VIP客户可以看到所有产品，优先显示VIP专属产品
            base_query += """ ORDER BY CASE 
                WHEN suitable_for_vip = 1 THEN 0 
                ELSE 1 
            END, category"""
        else:
            # 普通客户只看非VIP产品
            base_query += " AND suitable_for_vip = 0 ORDER BY category"
        
        # 限制结果数量
        base_query += f" LIMIT {max_results}"
        
        # 执行查询
        cursor.execute(base_query, tuple(query_params))
        recommended_products = cursor.fetchall()
        
        # 存储推荐结果到上下文
        context.context.query_results["recommended_products"] = recommended_products
        context.context.current_step = "完成推荐"
        context.context.workflow_complete = True
        
        if not recommended_products:
            return "没有找到符合客户条件的产品"
        
        # 构建筛选条件描述
        filter_desc = []
        if product_category:
            filter_desc.append(f"类别：{product_category}")
        if risk_level:
            filter_desc.append(f"风险等级：{risk_level}")
        
        filter_text = "（筛选：" + "、".join(filter_desc) + "）" if filter_desc else ""
        
        # 格式化推荐输出
        recommendations = f"为{customer_info['name']}({account_type}客户)推荐以下产品{filter_text}:\n\n"
        
        for i, product in enumerate(recommended_products, 1):
            interest_info = f"利率: {product['interest_rate']}%" if product['interest_rate'] is not None else ""
            term_info = f"期限: {product['term_months']}个月" if product['term_months'] > 0 else "无固定期限"
            
            recommendations += (
                f"{i}. {product['name']} ({product['category']})\n"
                f"   最低金额: {product['min_amount']} 元\n"
                f"   {interest_info}\n"
                f"   {term_info}\n"
                f"   风险等级: {product['risk_level']}\n\n"
            )
        
        # 根据客户类型添加特权信息
        if account_type == '白金':
            recommendations += "【白金会员特权】您可以享受产品手续费全免和专属理财经理服务。\n"
        elif account_type == 'VIP':
            recommendations += "【VIP会员特权】您可以享受产品手续费5折优惠。\n"
        
        return recommendations
    
    except Exception as e:
        return f"推荐产品时发生错误: {str(e)}"
    
    finally:
        if cursor:
            cursor.close()
        if connection:
            connection.close()

&emsp;&emsp;接下来，在定义产品推荐代理时，除了将`recommend_products`工具函数注册到产品推荐代理中以外，同时需要考虑：产品推荐代理需要依赖于客户信息助手代理，所以我们需要让产品推荐代理在执行过程中可以切换回客户信息助手代理。这就需要掌握`Agent`组件中的`handoffs`参数。

<style>
.center 
{
  width: auto;
  display: table;
  margin-left: auto;
  margin-right: auto;
}
</style>

<p align="center"><font face="黑体" size=4>Agent 组件核心参数</font></p>
<div class="center">

| 属性名                | 类型                                                                                          | 描述                                                                                                   |
|---------------------|-----------------------------------------------------------------------------------------------|--------------------------------------------------------------------------------------------------------|
| <font color=green>**name**</font>             | `str`                                                                                         | 代理的名称。                                                                                             |
| <font color=green>**instructions**</font>      | `str` \| `Callable[[RunContextWrapper[TContext], Agent[TContext]], MaybeAwaitable[str]]` \| `None` | 代理的指令，用作“系统提示”。可以是字符串或动态生成指令的函数。                                           |
| <font color=red>**handoff_description**</font> | `str` \| `None`                                                                              | 代理的描述，用于代理作为交接时，让 LLM 知道它的功能和何时调用它。                                         |
| <font color=red>**handoffs**</font>          | list[Agent[Any] | Handoff[TContext]]                                                   | 代理可以委托的子代理列表。允许关注点分离和模块化。                                                       |
| <font color=green>**model**</font>              | `str` \| `Model` \| `None`                                                                   | 调用 LLM 时使用的模型实现。默认情况下，如果未设置，代理将使用 `openai_provider.DEFAULT_MODEL` 中配置的默认模型。 |
| <font color=green>**model_settings**</font>    | `ModelSettings`                                                                               | 配置模型特定的调优参数（例如温度、top_p）。                                                             |
| <font color=green>**tools**</font>             | `list[Tool]`                                                                                 | 代理可以使用的工具列表。                                                                                 |
| `mcp_servers`       | `list[MCPServer]`                                                                             | 代理可以使用的模型上下文协议（MCP）服务器列表。                                                        |
| `mcp_config`        | `MCPConfig`                                                                                   | MCP 服务器的配置。                                                                                       |
| `input_guardrails`  | `list[InputGuardrail[TContext]]`                                                             | 在代理执行之前并行运行的检查列表，仅在代理是链中的第一个代理时运行。                                     |
| `output_guardrails` | `list[OutputGuardrail[TContext]]`                                                            | 在生成响应后对代理的最终输出运行的检查列表，仅在代理生成最终输出时运行。                                 |
| `output_type`       | type[Any] | AgentOutputSchemaBase | None                                                 | 输出对象的类型。如果未提供，输出将为 `str`。                                                             |
| `hooks`             | AgentHooks[TContext] | None                                                               | 接收代理生命周期事件回调的类。                                                                           |
| `tool_use_behavior` | Literal["run_llm_again", "stop_on_first_tool"] | StopAtTools | ToolsToFinalOutputFunction | 配置工具使用的处理方式。                                                                                 |
| `reset_tool_choice` | `bool`                                                                                       | 调用工具后是否将工具选择重置为默认值。默认为 `True`。确保代理不会进入工具使用的无限循环。                   |

&emsp;&emsp;接下来需要注意：`Agent`组件中的`handoffs`参数接收的类型为`Agent[Any] | Handoff[TContext]`，源码位置：https://github.com/openai/openai-agents-python/blob/main/src/agents/agent.py

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202505081747781.png" width=60%></div>     

&emsp;&emsp;所以我们有两种方式构建转接代理，第一种比较简单的方法是直接传递`Agent`对象，如下所示：

In [50]:
# 选择一个客户ID
# 从某个界面选择客户的过程
mysql_ctx = MySQLContext(
    connection_pool=pool,
    customer_id="CUST000100",  # 示例客户ID
)


recommend_agent = Agent[MySQLContext](
    name="产品推荐助手",
    instructions="请使用中文回答用户的问题",
    model=OpenAIChatCompletionsModel(
        model=DEEPSEEK_MODEL,
        openai_client=deepseek_client,
    ),
    model_settings=ModelSettings(
        temperature=0.6,
        max_tokens=2048,
    ),
    tools=[recommend_products],
    handoffs=[info_agent]  # 可以切换回客户信息助手，当然也可以传递多个代理
)


# 产品推荐
result4 = await Runner.run(
    starting_agent=recommend_agent,
    input="查询客户近一年的存款记录，推荐一个储蓄产品",
    context=mysql_ctx
)
print("\n===== 产品推荐 =====")
print(result4.final_output)


===== 产品推荐 =====
根据客户的交易记录和基本信息，以下是推荐和分析：

### 交易记录分析
- 最近一年内有一笔大额存款（542,800元）存入3个月定期存款产品。
- 当前账户余额为42,733元。

### 推荐储蓄产品
1. **大额存单（3个月或6个月）**
   - **理由**：客户已有大额存款习惯，且偏好短期产品。大额存单利率通常高于普通定期存款，适合短期资金增值。
   - **利率**：约2.5%-3.0%（具体以银行公告为准）。

2. **货币基金**
   - **理由**：客户当前账户余额较低，货币基金流动性高，适合随时存取，同时收益高于活期存款。
   - **预期年化收益**：约2.0%-2.5%。

3. **结构性存款（保本型）**
   - **理由**：结合定期存款和金融衍生品，收益浮动但保本，适合风险偏好较低的客户。
   - **预期收益**：1.5%-3.5%（根据市场情况浮动）。

### 建议
- 如果客户希望保持资金流动性，推荐货币基金。
- 如果客户有短期闲置资金，推荐大额存单或结构性存款。

如需进一步了解或办理，请提供更多需求细节！


&emsp;&emsp;这里是有问题的，细心一点就能发现，客户的基础信息确实查询到了，但是最终的产品推荐其实是大模型直接输出的，而不是通过工具函数`recommend_products`输出的，也就是生成了大模型的幻觉。我们可以查看中间过程的执行结结果来进行验证：

In [51]:
len(result4.new_items)

7

In [52]:
def format_item(item):
    """格式化单个 item 为易读的格式"""
    raw_item = item.raw_item
    
    if hasattr(raw_item, 'type') and raw_item.type == 'function_call':
        # 处理函数调用
        return {
            "类型": "函数调用",
            "函数名": raw_item.name,
            "调用ID": raw_item.call_id,
            "参数": json.loads(raw_item.arguments) if raw_item.arguments else {}
        }
    elif isinstance(raw_item, dict) and raw_item.get('type') == 'function_call_output':
        # 处理函数调用输出
        return {
            "类型": "函数调用输出",
            "调用ID": raw_item.get('call_id'),
            "输出": raw_item.get('output')
        }
    elif hasattr(raw_item, 'type') and raw_item.type == 'message':
        # 处理消息输出
        content = []
        for item in raw_item.content:
            if hasattr(item, 'text'):
                content.append(item.text)
        return {
            "类型": "消息",
            "角色": raw_item.role,
            "内容": "\n".join(content)
        }
    else:
        # 其他类型
        return {"原始数据": str(raw_item)}

In [53]:
# 输出美化版本
import json
from pprint import pprint

print("\n=== 格式化的代理交互流程 ===\n")

for i, item in enumerate(result4.new_items):
    print(f"\n== item {i+1} ==")
    formatted = format_item(item)
    pprint(formatted, width=100, sort_dicts=False)


=== 格式化的代理交互流程 ===


== item 1 ==
{'类型': '函数调用',
 '函数名': 'transfer_to_______',
 '调用ID': 'call_0_eed534cd-d57d-4351-a8d5-87652e983f66',
 '参数': {}}

== item 2 ==
{'类型': '函数调用输出',
 '调用ID': 'call_0_eed534cd-d57d-4351-a8d5-87652e983f66',
 '输出': "{'assistant': '客户信息助手'}"}

== item 3 ==
{'类型': '函数调用',
 '函数名': 'query_transaction_history',
 '调用ID': 'call_0_cdf11470-9387-4e97-a20e-d34c7c544d34',
 '参数': {'days': 365, 'transaction_type': '存款'}}

== item 4 ==
{'类型': '函数调用',
 '函数名': 'query_customer_info',
 '调用ID': 'call_1_c9d8bb5f-d621-42a9-b52b-443afb5bf857',
 '参数': {'include_sensitive_info': False, 'format_type': 'standard'}}

== item 5 ==
{'类型': '函数调用输出',
 '调用ID': 'call_0_cdf11470-9387-4e97-a20e-d34c7c544d34',
 '输出': '最近交易记录（筛选：最近365天、类型为存款）:\n'
       '1. 产品: 定期存款-3个月\n'
       '   金额: 542800.00 元\n'
       '   类型: 存款\n'
       '   日期: 2024-08-23T05:53:24\n'
       '   状态: 已完成\n'
       '\n'
       '共 1 笔交易，总金额: 542800.00 元'}

== item 6 ==
{'类型': '函数调用输出',
 '调用ID': 'call_1_c9d8bb5f-d621-42a9-b5

&emsp;&emsp;从中间过程看，`handoffs`其实会把交互代理当成工具调用来实现，当大模型决定需要进行交接时，它会标识为`transfer_to_x`，这个调用会被识别为一个 `HandoffCallItem`。

&emsp;&emsp;同时，也能够看到，在最终的输出结果中，并没有执行工具函数`recommend_products`，而是直接输出了结果。其原因就在于，当`recommend_agent`作为起始代理时，因为通过`handoffs`参数传递了`info_agent`，所以当用户的输入意图包含`info_agent`所负责的查询功能时，会先调用`info_agent`获取客户信息，但是当`info_agent`执行完毕后，因为在`info_agent` 代理我们并没有定义转接回`recommend_agent`的逻辑，所以大模型会直接输出最终结果，而不是继续调用`recommend_agent`，即没有形成闭环。因此，需要对`info_agent`代理进行改造，即在`info_agent`代理中定义转接回`recommend_agent`的逻辑，如下所示：


In [54]:
# 创建代理
info_agent = Agent[MySQLContext](
    name="客户信息助手",
    instructions="请使用中文回答用户的问题",
    model=OpenAIChatCompletionsModel(
        model=DEEPSEEK_MODEL,
        openai_client=deepseek_client,
    ),
    model_settings=ModelSettings(
        temperature=0.6,
        max_tokens=2048,
    ),
    tools=[query_customer_info, query_transaction_history], # 通过 tools 参数接收工具函数
    handoffs=[recommend_agent]  # 可以切换产品推荐助手
)



recommend_agent = Agent[MySQLContext](
    name="产品推荐助手",
    instructions="请使用中文回答用户的问题",
    model=OpenAIChatCompletionsModel(
        model=DEEPSEEK_MODEL,
        openai_client=deepseek_client,
    ),
    model_settings=ModelSettings(
        temperature=0.6,
        max_tokens=2048,
    ),
    tools=[recommend_products],
    handoffs=[info_agent]  # 可以切换回客户信息助手，当然也可以传递多个代理
)


&emsp;&emsp;再次进行调用测试：

In [60]:
# 选择一个客户ID
# 从某个界面选择客户的过程
mysql_ctx = MySQLContext(
    connection_pool=pool,
    customer_id="CUST000100",  # 示例客户ID
)


# 第三步：产品推荐
result5 = await Runner.run(
    starting_agent=recommend_agent,
    input="帮我查询客户的基本信息，然后看一下其近一年的存款记录，最终基于客户信息和交易历史推荐一个合适的储蓄产品",
    context=mysql_ctx
)
print("\n===== 产品推荐 =====")
print(result5.final_output)


===== 产品推荐 =====
根据客户何丹的基本信息和近一年的存款记录，以下是为其推荐的储蓄产品：

### 推荐产品：
1. **活期储蓄账户**
   - **类别**: 储蓄
   - **最低金额**: 0.00 元
   - **利率**: 0.35%
   - **期限**: 无固定期限
   - **风险等级**: 低

### 推荐理由：
- 客户何丹的账户类型为普通账户，且近期有一笔大额定期存款记录，表明其对储蓄产品有需求。
- 活期储蓄账户灵活性高，适合日常资金管理，同时利率稳定，风险低，符合客户的风险偏好。

如需进一步了解或办理，请随时联系！


In [61]:
# 输出美化版本
import json
from pprint import pprint

print("\n=== 格式化的代理交互流程 ===\n")

for i, item in enumerate(result5.new_items):
    print(f"\n== item {i+1} ==")
    formatted = format_item(item)
    pprint(formatted, width=100, sort_dicts=False)


=== 格式化的代理交互流程 ===


== item 1 ==
{'类型': '函数调用',
 '函数名': 'transfer_to_______',
 '调用ID': 'call_0_f074e8bb-8e20-4f84-b2a9-635f2cee0e41',
 '参数': {}}

== item 2 ==
{'类型': '函数调用输出',
 '调用ID': 'call_0_f074e8bb-8e20-4f84-b2a9-635f2cee0e41',
 '输出': "{'assistant': '客户信息助手'}"}

== item 3 ==
{'类型': '函数调用',
 '函数名': 'query_customer_info',
 '调用ID': 'call_0_b0cb6af8-5464-4adc-8367-73249767d59f',
 '参数': {'include_sensitive_info': False, 'format_type': 'standard'}}

== item 4 ==
{'类型': '函数调用',
 '函数名': 'query_transaction_history',
 '调用ID': 'call_1_e72f6418-13c6-4f63-a819-826e5eaf53dd',
 '参数': {'days': 365, 'transaction_type': '存款'}}

== item 5 ==
{'类型': '函数调用输出',
 '调用ID': 'call_0_b0cb6af8-5464-4adc-8367-73249767d59f',
 '输出': '客户信息:\n'
       '- 姓名: 何丹\n'
       '- 账户类型: 普通\n'
       '- 账户余额: 42733.00 元\n'
       '- 电话: ***********\n'
       '- 邮箱: ***********\n'
       '- 地址: 四川省刚市华龙阜新街D座 562358\n'
       '- 注册日期: 2022-03-01'}

== item 6 ==
{'类型': '函数调用输出',
 '调用ID': 'call_1_e72f6418-13c6-4f63-a819-826e5

&emsp;&emsp;从响应结果上看，本次交互的最终输出结果是`info_agent`和`recommend_agent`共同协作的结果，即`info_agent`负责查询客户信息，`recommend_agent`负责基于客户信息和交易历史推荐产品。同时也可以看`LocalContext`的`current_step`和`workflow_complete`参数，它们分别表示当前步骤和是否完成。如下所是：


In [62]:
# 打印最终查询结果状态
print("\n===== 工作流完成情况 =====")
print(f"当前步骤: {mysql_ctx.current_step}")
print(f"是否完成: {mysql_ctx.workflow_complete}")


===== 工作流完成情况 =====
当前步骤: 完成推荐
是否完成: True


&emsp;&emsp;`workflow_complete`为`True`，表示整个工作流已经完成。 

&emsp;&emsp;直接传递`Agent`对象时是最简单的构建多代理交接工作模式的方法。但是，大家可以通过上面的例子进行多次测试，其实解析效果是非常不稳定的。很多情况下都不能从`info_agent`代理中正确地切回到`recommend_agent`代理，而是会直接输出最终结果。主要原因还是通过`Agent`对象传递的灵活性还不够，不足以支撑复杂的代理协作。因此，生产环境下的应用往往需要采用第二种方式，即通过`handoff`函数来构建多代理协作。

&emsp;&emsp;`handoffs`参数用于定义代理可以委托的子代理列表。也就是说，通过`handoff`函数可以连接多个代理实现多代理协作。其源码定义位置：https://github.com/openai/openai-agents-python/blob/main/src/agents/handoffs.py


<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202505081731205.png" width=60%></div>     

&emsp;&emsp;`handoffs` 的本质就是通过`LocalContext`实现代理之间的上下文传递，当代理需要切换时，通过`handoffs`的实现机制将任务委派给对应的代理，并借助`LocalContext`保证在切换过程中上下文数据的一致，从而实现多代理之间的协作，即构建出 `Multi-Agent` 架构。这里我们梳理了`handoffs`支持传递的参数如下：


<style>
.center 
{
  width: auto;
  display: table;
  margin-left: auto;
  margin-right: auto;
}
</style>

<p align="center"><font face="黑体" size=4>Handoff 可应用的参数</font></p>
<div class="center">

| 参数名称                  | 类型                                   | 描述                                                                                     |
|---------------------------|----------------------------------------|------------------------------------------------------------------------------------------|
| <font color=red>**agent**</font>                 | `Agent[TContext]`                     | 要交接的目标代理。                                                                      |
| <font color=red>**tool_name_override**</font>      | str                                    | 用于重写表示交接的工具名称。                                                            |
| <font color=red>**tool_description_override**</font>| str                                   | 用于重写表示交接的工具描述。                                                            |
| `on_handoff`             | `OnHandoffWithInput[THandoffInput]`  | 当交接被调用时执行的函数，可以是带输入的处理函数或无输入的处理函数。                     |
| `input_type`             | `type[THandoffInput]`                 | 交接输入的类型。如果提供，输入将根据此类型进行验证。                                    |
| `input_filter`           | `Callable[[HandoffInputData], HandoffInputData]` | 过滤传递给下一个代理的输入数据的函数。默认情况下，新代理会看到整个对话历史。              |
| `strict_json_schema`      | `bool`                                | 输入 JSON 模式是否为严格模式。默认为 `True`，建议保持为 `True` 以提高输入的正确性。     |

&emsp;&emsp;这里我们先关注`agent`、`tool_name_override`和`tool_description_override`这三个参数，如果想进一步提升交接代理的准确性，优先考虑优化的点就是：

1. 明确指示先使用自己的工具
2. 强调只有在确实需要更多信息时才使用handoff
3. 要求获取信息后回到自己的工具，避免直接给出回答

In [63]:
# 创建代理
info_agent = Agent[MySQLContext](
    name="客户信息助手",
    instructions="请使用中文回答用户的问题",
    model=OpenAIChatCompletionsModel(
        model=DEEPSEEK_MODEL,
        openai_client=deepseek_client,
    ),
    model_settings=ModelSettings(
        temperature=0.6,
        max_tokens=2048,
    ),
    tools=[query_customer_info, query_transaction_history], # 通过 tools 参数接收工具函数
    handoffs=[handoff(
        agent=recommend_agent,
        tool_name_override="transfer_to_recommend_agent",
        tool_description_override="当且仅当用户明确需要产品推荐，且您已通过query_customer_info和query_transaction_history工具获取了足够的客户基本信息和交易历史后，才使用此工具将对话转交给产品推荐助手。切勿在客户信息不完整时使用此工具。",
        )]   
)



recommend_agent = Agent[MySQLContext](
    name="产品推荐助手",
    instructions="请使用中文回答用户的问题",
    model=OpenAIChatCompletionsModel(
        model=DEEPSEEK_MODEL,
        openai_client=deepseek_client,
    ),
    model_settings=ModelSettings(
        temperature=0.6,
        max_tokens=2048,
    ),
    tools=[recommend_products],
    handoffs = [handoff(
        agent=info_agent,
        tool_name_override="transfer_to_info_agent",
        tool_description_override="当且仅当您需要获取更多客户信息或查询历史交易记录且当前上下文中没有这些信息时，才使用此工具。使用此工具前，请明确说明您需要查询的具体信息类型（如基本资料、交易历史等）。获取信息后，您应当回到产品推荐流程。",
    )]
)

&emsp;&emsp;接下来进行测试运行：


In [66]:
# 选择一个客户ID
# 从某个界面选择客户的过程
mysql_ctx = MySQLContext(
    connection_pool=pool,
    customer_id="CUST000100",  # 示例客户ID
)


# 第三步：产品推荐
result5 = await Runner.run(
    starting_agent=recommend_agent,
    input="帮我查询客户的基本信息，看一下其近一年的存款记录，然后基于客户信息和交易历史推荐一个合适的储蓄产品",
    context=mysql_ctx
)
print("\n===== 产品推荐 =====")
print(result5.final_output)


===== 产品推荐 =====
### 客户基本信息
- **姓名**: 何丹  
- **账户类型**: 普通  
- **账户余额**: 42,733.00 元  
- **注册日期**: 2022-03-01  
- **客户价值评估**: 一般  

### 近一年存款记录
- **最近一笔存款**:  
  - **产品**: 定期存款-3个月  
  - **金额**: 542,800.00 元  
  - **日期**: 2024-08-23  

### 推荐储蓄产品
基于客户信息和交易历史，推荐以下储蓄产品：  
1. **活期储蓄账户**  
   - **类别**: 储蓄  
   - **最低金额**: 0.00 元  
   - **利率**: 0.35%  
   - **期限**: 无固定期限  
   - **风险等级**: 低  

如需进一步了解或办理，请随时告知！


In [67]:
# 打印最终查询结果状态
print("\n===== 工作流完成情况 =====")
print(f"当前步骤: {mysql_ctx.current_step}")
print(f"是否完成: {mysql_ctx.workflow_complete}")


===== 工作流完成情况 =====
当前步骤: 完成推荐
是否完成: True


In [68]:
# 输出美化版本
import json
from pprint import pprint

print("\n=== 格式化的代理交互流程 ===\n")

for i, item in enumerate(result5.new_items):
    print(f"\n== item {i+1} ==")
    formatted = format_item(item)
    pprint(formatted, width=100, sort_dicts=False)


=== 格式化的代理交互流程 ===


== item 1 ==
{'类型': '函数调用',
 '函数名': 'transfer_to_info_agent',
 '调用ID': 'call_0_9b8c02fc-540a-416a-a744-da47c08123e6',
 '参数': {}}

== item 2 ==
{'类型': '函数调用输出',
 '调用ID': 'call_0_9b8c02fc-540a-416a-a744-da47c08123e6',
 '输出': "{'assistant': '客户信息助手'}"}

== item 3 ==
{'类型': '函数调用',
 '函数名': 'query_customer_info',
 '调用ID': 'call_0_453c4b62-1044-465a-8a0f-abaf4fd64730',
 '参数': {'include_sensitive_info': False, 'format_type': 'detailed'}}

== item 4 ==
{'类型': '函数调用',
 '函数名': 'query_transaction_history',
 '调用ID': 'call_1_dbf38b28-2a16-4845-be64-ca9f0ab4c9c4',
 '参数': {'days': 365, 'transaction_type': '存款'}}

== item 5 ==
{'类型': '函数调用输出',
 '调用ID': 'call_0_453c4b62-1044-465a-8a0f-abaf4fd64730',
 '输出': '客户详细信息（ID: CUST000100）:\n'
       '- 姓名: 何丹\n'
       '- 账户类型: 普通\n'
       '- 账户余额: 42733.00 元\n'
       '- 电话: ***********\n'
       '- 邮箱: ***********\n'
       '- 地址: 四川省刚市华龙阜新街D座 562358\n'
       '- 注册日期: 2022-03-01\n'
       '- 客户价值评估: 一般'}

== item 6 ==
{'类型': '函数调用输出',


&emsp;&emsp;经过测试，使用`handoff`方法传递交接代理时，在当前场景下能直接把`Workflow`的正确执行流程准确率提升到95%以上。大家可以根据上述的代码进行验证。当然，针对不同的场景，我们需要灵活的调整`handoff`的参数，并要结合工具函数的描述（Json Schema）综合进行优化，才能达到一个比较稳定的执行效果。而通过`handoff`方法构建多智能体协作的方式，也是建议大家在生产环境中优先考虑的方案。


&emsp;&emsp;通过上述示例，能够明显感觉到`Local Context`的关键优势主要就在于通过状态持久化提供了资源共享、工具间通信、多代理协作等一系列数据支撑，是`OpenAI Agents SDK`中实现复杂应用程序的关键机制，而本质上，`Handoffs` 就是利用了上下文管理机制才实现的多个 `Agent` 之间的交互，例如：第一个 `Agent` 查询了客户信息并存储在上下文中，第二个 `Agent` 可以直接访问这些信息而不需要重新查询。同时，`Handoffs` 会表现为大模型可以调用的工具，正如我们上面看到的：`transfer_to_info_agent` 和 `transfer_to_recommend_agent`。

&emsp;&emsp;


### 3.2 Model Context

&emsp;&emsp;理解了`Agent Local Context`后，`Model Context`就非常容易理解了，该上下文类型指的是：在生成响应时能够看到的数据，即对话历史中的内容。最常应用的一种方式就是在构建`Agent`时，通过`instructions`进行详细的任务描述。这里需要了解的一个细节是：构建`Handoffs`时，官方定义了这样的一个提示前缀来进行提示：


```python
    RECOMMENDED_PROMPT_PREFIX = (
    "# System context\n"
    "You are part of a multi-agent system called the Agents SDK, designed to make agent "
    "coordination and execution easy. Agents uses two primary abstraction: **Agents** and "
    "**Handoffs**. An agent encompasses instructions and tools and can hand off a "
    "conversation to another agent when appropriate. "
    "Handoffs are achieved by calling a handoff function, generally named "
    "`transfer_to_<agent_name>`. Transfers between agents are handled seamlessly in the background;"
    " do not mention or draw attention to these transfers in your conversation with the user.\n"
)
```

&emsp;&emsp;我们翻译成中文如下所示：

```markdown
    # 系统上下文
    您是一个名为 Agents SDK 的多代理系统的一部分，旨在简化代理的协调和执行。代理使用两个主要抽象：**Agents**和**Handoff**。一个代理包含指令和工具，并可以在适当的时候将对话交接给另一个代理。
    交接是通过调用一个交接函数来实现的，通常命名为 `transfer_to_<agent_name>`。代理之间的转移在后台无缝处理；请不要在与用户的对话中提及或引起对这些转移的注意。
```

&emsp;&emsp;所以一种比较有效的优化方式是：在该系统提示词前缀之后，进一步添加代理代理的行为描述，如下所示：

In [69]:
RECOMMENDED_PROMPT_PREFIX = (
    "# 系统上下文\n"
    "您是一个名为 Agents SDK 的多代理系统的一部分，旨在简化代理的协调和执行。"
    "代理使用两个主要抽象：**Agents** 和 **Handoff**。一个代理包含指令和工具，并可以在适当的时候将对话交接给另一个代理。\n"
    "交接是通过调用一个交接函数来实现的，通常命名为 `transfer_to_<agent_name>`。代理之间的转移在后台无缝处理；请不要在与用户的对话中提及或引起对这些转移的注意。\n"
)


# 1. 首先创建recommend_agent，s
recommend_agent = Agent[MySQLContext](
    name="产品推荐助手",
    instructions=f"""{RECOMMENDED_PROMPT_PREFIX}
    您是一位专业的金融产品推荐专家，擅长根据客户信息和交易历史推荐合适的金融产品。

    请按照以下顺序处理用户请求：
    1. 首先尝试使用您自己的工具(recommend_products)基于已有信息推荐产品
    2. 只有在确实需要更多客户信息或交易历史且当前上下文中没有这些信息时，才使用transfer_to_info_agent工具
    3. 一旦获得所需的客户信息，请回到自己的工具推荐产品，不要直接给出回答

    请使用中文与用户交流，提供专业的产品推荐。
    """,
    model=OpenAIChatCompletionsModel(
        model=DEEPSEEK_MODEL,
        openai_client=deepseek_client,
    ),
    model_settings=ModelSettings(
        temperature=0.6,
        max_tokens=2048,
    ),
    tools=[recommend_products],
    handoffs = [handoff(
        agent=info_agent,
        tool_name_override="transfer_to_info_agent",
        tool_description_override="当且仅当您需要获取更多客户信息或查询历史交易记录且当前上下文中没有这些信息时，才使用此工具。使用此工具前，请明确说明您需要查询的具体信息类型（如基本资料、交易历史等）。获取信息后，您应当回到产品推荐流程。",
    )]
)

# 2. 然后创建info_agent，包含对recommend_agent的handoff
info_agent = Agent[MySQLContext](
    name="客户信息助手",
    instructions=f"""{RECOMMENDED_PROMPT_PREFIX}
    您是一位专业的客户信息查询助手，擅长检索和提供客户详细信息及交易历史记录。

    请按照以下顺序处理用户请求：
    1. 首先使用您自己的工具(query_customer_info, query_transaction_history)获取客户信息
    2. 如果用户明确表示需要产品推荐，且您已获取到足够的客户信息后，才使用transfer_to_recommend_agent工具
    3. 切勿在未获取充分客户信息的情况下过早转交给产品推荐助手

    请使用中文与用户交流，确保提供准确的客户信息。
    """,
    model=OpenAIChatCompletionsModel(
        model=DEEPSEEK_MODEL,
        openai_client=deepseek_client,
    ),
    model_settings=ModelSettings(
        temperature=0.6,
        max_tokens=2048,
    ),
    tools=[query_customer_info, query_transaction_history],
    handoffs=[handoff(
        agent=recommend_agent,
        tool_name_override="transfer_to_recommend_agent",
        tool_description_override="当且仅当用户明确需要产品推荐，且您已通过query_customer_info和query_transaction_history工具获取了足够的客户基本信息和交易历史后，才使用此工具将对话转交给产品推荐助手。切勿在客户信息不完整时使用此工具。",
        )]  
)

In [70]:
# 选择一个客户ID
# 从某个界面选择客户的过程
mysql_ctx = MySQLContext(
    connection_pool=pool,
    customer_id="CUST000100",  # 示例客户ID
)


# 第三步：产品推荐
result5 = await Runner.run(
    starting_agent=recommend_agent,
    input="帮我查询客户的基本信息，看一下其近一年的存款记录，然后基于客户信息和交易历史推荐一个合适的储蓄产品",
    context=mysql_ctx
)
print("\n===== 产品推荐 =====")
print(result5.final_output)


===== 产品推荐 =====
根据客户何丹的基本信息和近一年的存款记录，以下是为其推荐的储蓄产品：

### 推荐产品：
1. **活期储蓄账户**
   - **类别**: 储蓄
   - **最低金额**: 0.00 元
   - **利率**: 0.35%
   - **期限**: 无固定期限
   - **风险等级**: 低

### 推荐理由：
- 客户近一年有一笔大额定期存款记录，表明其有储蓄需求。
- 活期储蓄账户灵活且无门槛，适合客户作为日常资金管理工具。
- 低风险产品符合客户的普通账户类型和价值评估。

如果需要进一步了解其他产品或调整推荐条件，请随时告知！


In [71]:
# 打印最终查询结果状态
print("\n===== 工作流完成情况 =====")
print(f"当前步骤: {mysql_ctx.current_step}")
print(f"是否完成: {mysql_ctx.workflow_complete}")


===== 工作流完成情况 =====
当前步骤: 完成推荐
是否完成: True


In [72]:
# 输出美化版本
import json
from pprint import pprint

print("\n=== 格式化的代理交互流程 ===\n")

for i, item in enumerate(result5.new_items):
    print(f"\n== item {i+1} ==")
    formatted = format_item(item)
    pprint(formatted, width=100, sort_dicts=False)


=== 格式化的代理交互流程 ===


== item 1 ==
{'类型': '函数调用',
 '函数名': 'transfer_to_info_agent',
 '调用ID': 'call_0_bd8c09c0-13e4-48f2-a191-5d181c422a31',
 '参数': {}}

== item 2 ==
{'类型': '函数调用输出',
 '调用ID': 'call_0_bd8c09c0-13e4-48f2-a191-5d181c422a31',
 '输出': "{'assistant': '客户信息助手'}"}

== item 3 ==
{'类型': '函数调用',
 '函数名': 'query_customer_info',
 '调用ID': 'call_0_50915a6b-67a5-4e73-b33a-18158b1f021a',
 '参数': {'include_sensitive_info': False, 'format_type': 'detailed'}}

== item 4 ==
{'类型': '函数调用',
 '函数名': 'query_transaction_history',
 '调用ID': 'call_1_b16e6f7e-eb29-4f42-8a39-dddb834ef0c9',
 '参数': {'days': 365, 'transaction_type': '存款'}}

== item 5 ==
{'类型': '函数调用输出',
 '调用ID': 'call_0_50915a6b-67a5-4e73-b33a-18158b1f021a',
 '输出': '客户详细信息（ID: CUST000100）:\n'
       '- 姓名: 何丹\n'
       '- 账户类型: 普通\n'
       '- 账户余额: 42733.00 元\n'
       '- 电话: ***********\n'
       '- 邮箱: ***********\n'
       '- 地址: 四川省刚市华龙阜新街D座 562358\n'
       '- 注册日期: 2022-03-01\n'
       '- 客户价值评估: 一般'}

== item 6 ==
{'类型': '函数调用输出',


In [76]:
# 选择一个客户ID
# 从某个界面选择客户的过程
mysql_ctx = MySQLContext(
    connection_pool=pool,
    customer_id="CUST000100",  # 示例客户ID
)


# 第三步：产品推荐
result5 = await Runner.run(
    starting_agent=recommend_agent,
    input="根据客户近两年存款记录推荐一款储蓄产品",
    context=mysql_ctx
)
print("\n===== 产品推荐 =====")
print(result5.final_output)


===== 产品推荐 =====
根据客户何丹近两年的存款记录（主要为定期存款），我们推荐以下储蓄产品：

### 推荐产品：活期储蓄账户
- **类别**：储蓄  
- **最低金额**：0.00 元  
- **利率**：0.35%  
- **特点**：无固定期限，资金灵活存取  
- **风险等级**：低  

这款产品适合需要灵活管理资金的客户，同时风险较低。如果需要其他类型的储蓄产品或更多选择，请随时告知！


&emsp;&emsp;以上就是本节课程的全部内容，除此外，`OpenAI Agents-SDK`框架中安全护栏、Trace以及接入MCP构建智能体相关的内容和实战，我们在下一节课再展开详细的介绍。